In [1]:
import numpy as np
import pandas as pd
import math
import os
from datetime import datetime
from tqdm import tqdm

In [2]:
def has_aki(urine, weight):
    value = urine/(weight*6)
    if value <= 0.5:
        return 1
    else:
        return 2
    
def one_hour_pass(time_2, time_3):
    if time_3 - time_2 >= 3600:
        return 1
    else:
        return 0

def list_avg(list):
    value = 0
    for item in list:
        value += item
    return (value/len(list))

In [3]:
urine_path = '../raw_data/urine/'
urine_filelist = os.listdir(urine_path)
########for MAC#####################

#urine_filelist.remove('.DS_Store')#

####################################
print(urine_filelist)

['201401-6h.xlsx', '201402-6h.xlsx', '201403-6h.xlsx', '201404-6h.xlsx', '201405-6h.xlsx', '201406-6h.xlsx', '201407-6h.xlsx', '201408-6h.xlsx', '201409-6h.xlsx', '201410-6h.xlsx', '201411-6h.xlsx', '201412-6h.xlsx', '201501-6h.xlsx', '201502-6h.xlsx', '201503-6h.xlsx', '201504-6h.xlsx', '201505-6h.xlsx', '201506-6h.xlsx', '201507-6h.xlsx', '201508-6h.xlsx', '201509-6h.xlsx', '201510-6h.xlsx', '201511-6h.xlsx', '201512-6h.xlsx', '201601-6h.xlsx', '201602-6h.xlsx', '201603-6h.xlsx', '201604-6h.xlsx', '201605-6h.xlsx', '201606-6h.xlsx', '201607-6h.xlsx', '201608-6h.xlsx', '201609-6h.xlsx', '201610-6h.xlsx', '201611-6h.xlsx', '201612-6h.xlsx', '201701-6h.xlsx', '201702-6h.xlsx', '201703-6h.xlsx', '201704-6h.xlsx', '201705-6h.xlsx', '201706-6h.xlsx', '201707-6h.xlsx', '201708-6h.xlsx', '201709-6h.xlsx', '201710-6h.xlsx', '201711-6h.xlsx', '201712-6h.xlsx', '201801-6h.xlsx', '201802-6h.xlsx', '201803-6h.xlsx', '201804-6h.xlsx', '201805-6h.xlsx', '201806-6h.xlsx', '201807-6h.xlsx', '201808-6

In [4]:
big_patient_list = []
big_label_list = []
big_start_time_list = []
big_end_time_list = []
big_age_list = []
big_weight_list = []
big_base_list = []
big_icu_time_list = []
big_leave_time_list = []

### 單一時間

In [5]:
time_interval  = 35
time_interval_str = str(time_interval)
time_in_second = (time_interval - 1) * 3600

In [6]:
aki = 0
no_aki = 0
month = 1
for FILE in tqdm(urine_filelist):
#     print(month)
    month += 1
    df = pd.read_excel(urine_path + FILE, sheet_name = "匯出資料", usecols = ["6小時總尿量", "尿液輸出前最新體重", "尿液輸出時間", "病歷號", "年齡"]) 
    #6小時總尿量, 體重, 輸出時間, 病歷號 年齡
    df = df.to_numpy()
    patient_list = [0]
    label_list = [0] 
    index = 0
    weight_list = [0]
    cur_weight_list = []
    cur_urine_list = []
    start_time_list = [0]
    end_time_list = [0]
    age_list = [0]
    base_list = [0]
    icu_time_list = [0]
    leave_time_list = [0]
    label = 3 #label: 0 沒有AKI; 1 有AKI, 2 入院23小時內就得AKI; 3 資料有缺失
    time_1 = 0 #進ICU的時間
    time_2 = 0 #一個小時的起始點
    time_3 = 0 #當前紀錄的時間點
    time_4 = 0 #重要的時間點 label 0 不會用到 label 1 等於發病時間前一小時
    last_patient = 0
    cur_patient = 0
    ary_list = []
    weight_assumption = False
    age = False
    
    for num_of_records in range(0,df.shape[0]):
        ########碰到x病人就直接跳過
        if str.isdigit(str(df[num_of_records][3])) == False:
            continue
        ########碰到年齡有問題的病人也跳過
        elif (df[num_of_records][4] < 18 or np.isnan(df[num_of_records][4]) == 1):
            continue
        ########碰到新的病人
        elif int(df[num_of_records][3]) != cur_patient:
            #print(cur_patient)
        ########先處裡舊的 df的第一個病人不會有舊的病人可以處理    
            if cur_patient != last_patient:
                #print('process')
                ########開始處理
                age = ary_list[0][4]
                ########先取得病人最早的體重紀錄 以防有尿量紀錄卻無體重紀錄的狀況
                weight_assumption = False
                #print(ary_list)
                for temp in ary_list:
                    if weight_assumption == False and np.isnan(temp[1]) == 0 and temp[1] != 0:
                        weight_assumption = temp[1]
                        break
                        
                ########確定病人有體重紀錄
                if weight_assumption != False:
                    ########清空變數         
                    cur_weight_list = []
                    cur_urine_list = []
                    time_1 = 0
                    time_2 = 0
                    time_3 = 0
                    last_time_3 = 0
                    time_4 = 0
                    label = 3

                    ########初始化變數
                    time_1 = ary_list[0][2].timestamp()
                    time_2 = time_1
                    
                    ########開始label
                    for temp in ary_list:        
                        time_3 = temp[2].timestamp()
                        cur_label = False
                        
                        ########處裡前幾筆資料的label
                        if one_hour_pass(time_2, time_3) == 1 and label != 2 and label != 1:
                            if len(cur_urine_list) and len(cur_weight_list) != 0:
                                cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
                            elif len(cur_urine_list) != 0:
                                cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
                            ########超過一小時 但前幾筆資料沒有6小時尿量紀錄
                            else:
                                cur_label = False
                            #print(cur_label)
                            ########
                            if cur_label != False:
                                if cur_label == 1:
                                    if time_2 - time_1 >= time_in_second:
                                        label = 1
                                        time_4 = time_2
                                    else: 
                                        label = 2
                                else:
                                    label = 0
                                #print(label)

                            ########調整基準時間 計算前後兩筆紀錄是否超過6小時
                            if (time_3 - last_time_3 > 21600) and (label != 1) and (label != 2) and (last_time_3 != 0):
                                if time_2 + 21600 - time_1 >= time_in_second:
                                    label = 1
                                    time_4 = time_2 + 21600 
                                else:
                                    label = 2
                                    
                            while time_3 - time_2 >= 3600:
                                time_2 += 3600        
                                    
                            ########清空尿量和體重暫存器
                            cur_weight_list = []
                            cur_urine_list = []
                        ########累積尿量及體重資料
                        if np.isnan(temp[0]) == 0:
                            cur_urine_list.append(temp[0])

                        if np.isnan(temp[1]) == 0 and temp[1] != 0:
                            cur_weight_list.append(temp[1])
                            
                        ########儲存這次紀錄的時間 之後和下次比對    
                        last_time_3 = time_3    

                    ########ary_list循環完畢 但urine_list不是空的 => 最後未滿一小時的資料還沒進行label        
                    if len(cur_urine_list) != 0 and label != 2 and label != 1:
                        if len(cur_urine_list) and len(cur_weight_list) != 0:
                            cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
                        elif len(cur_urine_list) != 0:
                            cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
                        ########if anything happens
                        else:
                            cur_label = False
                        ########
                        if cur_label != False:
                            if cur_label == 1:
                                ######## 86400為24小時
                                # if time_2 - time_1 >= 86400:
                                #     label = 1
                                #     time_4 = time_2 - 3600
                                ######## 43200為12小時
                                if time_2 - time_1 >= time_in_second:
                                    label = 1
                                    time_4 = time_2 
                                else: 
                                    label = 2
                            else:
                                label = 0
                    ########把該append進去處理好的list的資料append進去
                    #print(label)
                    if label == 1:
                        if weight_assumption > 15:
                            patient_list.append(cur_patient)
                            aki += 1
                            end_time_list.append(time_4)
                            start_time_list.append(time_4 - time_in_second)
                            label_list.append(label)
                            age_list.append(age)
                            weight_list.append(weight_assumption)
                            base_list.append('u')
                            icu_time_list.append(time_1)
                            leave_time_list.append(time_3)
                    elif label == 0:
                        if weight_assumption > 15:
                            if time_2 - time_1 >= time_in_minute:
                                patient_list.append(cur_patient)
                                no_aki += 1
                                end_time_list.append(time_2)
                                start_time_list.append(time_2 - time_in_second)
                                label_list.append(label)
                                age_list.append(age)
                                weight_list.append(weight_assumption)
                                base_list.append('u')
                                icu_time_list.append(time_1)
                                leave_time_list.append(time_3)
                    ########結束處理 
            ########更新現今處理病人的病歷號
            last_patient = cur_patient
            cur_patient = int(df[num_of_records][3])
            ########把新的病人append進來
            ary_list = []
            ary_list.append(df[num_of_records])
        ########持續append cur_patient的資料
        else:
            #print('append')
            ary_list.append(df[num_of_records])                        
                        
    ########處理最後一個病人
    age = ary_list[0][4]
    weight_assumption = False
    for temp in ary_list:
        if weight_assumption == False and np.isnan(temp[1]) == 0 and temp[1] != 0:
            weight_assumption = temp[1]
            break
            
    ########確定病人有體重紀錄        
    if weight_assumption != False:
        ########清空變數         
        cur_weight_list = []
        cur_urine_list = []
        time_1 = 0
        time_2 = 0
        time_3 = 0
        last_time_3 = 0
        time_4 = 0
        label = 3

        ########初始化變數
        time_1 = ary_list[0][2].timestamp()
        time_2 = time_1

        ########開始label
        for temp in ary_list:        
            time_3 = temp[2].timestamp()
            ########處裡前幾筆資料的label
            if one_hour_pass(time_2, time_3) == 1 and label != 2 and label != 1:
                if len(cur_urine_list) and len(cur_weight_list) != 0:
                    cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
                elif len(cur_urine_list) != 0:
                    cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
                ########超過一小時 但前幾筆資料沒有6小時尿量紀錄
                else:
                    cur_label = False
                ########
                if cur_label != False:
                    if cur_label == 1:
                        if time_2 - time_1 >= time_in_second:
                            label = 1
                            time_4 = time_2
                        else: 
                            label = 2
                    else:
                        label = 0

                ########調整基準時間 計算前後兩筆紀錄是否超過6小時
                if (time_3 - last_time_3 > 21600) and (label != 1) and (label != 2) and (last_time_3 != 0):
                    if time_2 + 21600 - time_1 >= time_in_second:
                        label = 1
                        time_4 = time_2 + 21600 
                    else:
                        label = 2
                        
                while time_3 - time_2 >= 3600:
                    time_2 += 3600
             
                ########清空尿量和體重暫存器
                cur_weight_list = []
                cur_urine_list = []
            ########累積尿量及體重資料
            if np.isnan(temp[0]) == 0:
                cur_urine_list.append(temp[0])

            if np.isnan(temp[1]) == 0 and temp[1] != 0:
                cur_weight_list.append(temp[1])
                
            ########儲存這次紀錄的時間 之後和下次比對    
            last_time_3 = time_3
        ########ary_list循環完畢 但urine_list不是空的 => 最後未滿一小時的資料還沒進行label        
        if len(cur_urine_list) != 0 and label != 2 and label != 1:
            if len(cur_urine_list) and len(cur_weight_list) != 0:
                cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
            elif len(cur_urine_list) != 0:
                cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
            ########if anything happens
            else:
                cur_label = False
            ########
            if cur_label != False:
                if cur_label == 1:
                    if time_2 - time_1 >= time_in_second:
                        label = 1
                        time_4 = time_2
                    else: 
                        label = 2
                else:
                    label = 0
    ########單一FILE所有病人處理完畢
    
        ########把該append進去處理好的list的資料append進去
        if label == 1:
            if weight_assumption > 15:
                patient_list.append(cur_patient)
                aki += 1
                end_time_list.append(time_4)
                start_time_list.append(time_4 - time_in_second)
                label_list.append(label)
                age_list.append(age)
                weight_list.append(weight_assumption)
                base_list.append('u')
                icu_time_list.append(time_1)
                leave_time_list.append(time_3)
        elif label == 0:
            if weight_assumption > 15:
                if time_2 - time_1 >= time_in_minute:
                    patient_list.append(cur_patient)
                    no_aki += 1
                    end_time_list.append(time_2)
                    start_time_list.append(time_2 - time_in_second)
                    label_list.append(label)
                    age_list.append(age)
                    weight_list.append(weight_assumption)
                    base_list.append('u')
                    icu_time_list.append(time_1)
                    leave_time_list.append(time_3)
            
    ########整理小list    
    patient_list.pop(0)
    label_list.pop(0)
    start_time_list.pop(0)
    end_time_list.pop(0)
    age_list.pop(0)
    weight_list.pop(0)
    base_list.pop(0)
    icu_time_list.pop(0)
    leave_time_list.pop(0)
    
    ########放進大list
    for value in patient_list:
        big_patient_list.append(value)
    for value in label_list:
        big_label_list.append(value)
    for value in start_time_list:
        big_start_time_list.append(value)
    for value in end_time_list:
        big_end_time_list.append(value)
    for value in age_list:
        big_age_list.append(value)
    for value in weight_list:
        big_weight_list.append(value)
    for value in base_list:
        big_base_list.append(value)
    for value in icu_time_list:
        big_icu_time_list.append(value)
    for value in leave_time_list:
        big_leave_time_list.append(value)
        
    #label的結果
    #print(patient_list)        
    #print(label_list)
    #print(len(patient_list))
    #print(len(label_list))
    #print(FILE)
    #print(age_list)
#     break
# print(df)
    
print(aki)
print(no_aki)

100%|██████████████████████████████████████████████████████████████████████████████████| 76/76 [04:05<00:00,  3.22s/it]

3274
2616


In [7]:
big_patient_list = list(map(int, big_patient_list))

In [8]:
patient_frame = pd.DataFrame(big_patient_list, columns = ['patient_code'])
patient_frame = pd.concat([patient_frame, pd.DataFrame(big_label_list, columns = ['label'])], axis = 1)
patient_frame = pd.concat([patient_frame, pd.DataFrame(big_start_time_list, columns = ['start_time'])], axis = 1)
patient_frame = pd.concat([patient_frame, pd.DataFrame(big_end_time_list, columns = ['end_time'])], axis = 1)
patient_frame = pd.concat([patient_frame, pd.DataFrame(big_age_list, columns = ['age'])], axis = 1)
patient_frame = pd.concat([patient_frame, pd.DataFrame(big_weight_list, columns = ['weight'])], axis = 1)
patient_frame = pd.concat([patient_frame, pd.DataFrame(big_base_list, columns = ['base'])], axis = 1)
patient_frame = pd.concat([patient_frame, pd.DataFrame(big_icu_time_list, columns = ['icu_time'])], axis = 1)
patient_frame = pd.concat([patient_frame, pd.DataFrame(big_leave_time_list, columns = ['leave_time'])], axis = 1)


In [9]:
if not os.path.exists('../processing_data/label1/' + time_interval_str):
    os.mkdir('../processing_data/label1/' + time_interval_str)
patient_frame.to_csv('../processing_data/label1/' + time_interval_str + '/patient_list.csv', index = False, encoding = 'utf-8')

In [10]:
#delete 18934939 manually => age is nan
print(len(big_age_list))
print(len(big_start_time_list))
print(len(big_patient_list))

5890
5890
5890


### 複合時間

In [11]:
time_interval_upper  = 28
time_interval_upper_str = str(time_interval_upper)
time_in_second_upper = (time_interval_upper - 1) * 3600
time_interval_lower  = 28
time_interval_lower_str = str(time_interval_lower)
time_in_second_lower = (time_interval_lower - 1) * 3600

In [12]:
##########################################################################################
#                  //       //////             //////        //                          #
#                 //      //   //            //   //        //                           #
#                //          //      /////      //         //  //                        #
#               //         //                 //          /////////                      #
#              //        ///////            ////////          //                         #
##########################################################################################
aki = 0
no_aki = 0
month = 1
for FILE in tqdm(urine_filelist):
#     print(month)
    month += 1
    df = pd.read_excel(urine_path + FILE, sheet_name = "匯出資料", usecols = ["6小時總尿量", "尿液輸出前最新體重", "尿液輸出時間", "病歷號", "年齡"]) 
    #6小時總尿量, 體重, 輸出時間, 病歷號 年齡
    df = df.to_numpy()
    patient_list = [0]
    label_list = [0] 
    index = 0
    weight_list = [0]
    cur_weight_list = []
    cur_urine_list = []
    start_time_list = [0]
    end_time_list = [0]
    age_list = [0]
    base_list = [0]
    icu_time_list = [0]
    leave_time_list = [0]
    label = 3 #label: 0 沒有AKI; 1 有AKI, 2 入院23小時內就得AKI; 3 資料有缺失
    time_1 = 0 #進ICU的時間
    time_2 = 0 #一個小時的起始點
    time_3 = 0 #當前紀錄的時間點
    time_4 = 0 #重要的時間點 label 0 不會用到 label 1 等於發病時間前一小時
    last_patient = 0
    cur_patient = 0
    ary_list = []
    weight_assumption = False
    age = False
    
    for num_of_records in range(0,df.shape[0]):
        ########碰到x病人就直接跳過
        if str.isdigit(str(df[num_of_records][3])) == False:
            continue
        ########碰到年齡有問題的病人也跳過
        elif (df[num_of_records][4] < 18 or np.isnan(df[num_of_records][4]) == 1):
            continue
        ########碰到新的病人
        elif int(df[num_of_records][3]) != cur_patient:
            #print(cur_patient)
        ########先處裡舊的 df的第一個病人不會有舊的病人可以處理    
            if cur_patient != last_patient:
                #print('process')
                ########開始處理
                age = ary_list[0][4]
                ########先取得病人最早的體重紀錄 以防有尿量紀錄卻無體重紀錄的狀況
                weight_assumption = False
                #print(ary_list)
                for temp in ary_list:
                    if weight_assumption == False and np.isnan(temp[1]) == 0 and temp[1] != 0:
                        weight_assumption = temp[1]
                        break
                        
                ########確定病人有體重紀錄
                if weight_assumption != False:
                    ########清空變數         
                    cur_weight_list = []
                    cur_urine_list = []
                    time_1 = 0
                    time_2 = 0
                    time_3 = 0
                    last_time_3 = 0
                    time_4 = 0
                    label = 3

                    ########初始化變數
                    time_1 = ary_list[0][2].timestamp()
                    time_2 = time_1
                    ########開始label
                    #print('start_label')
                    for temp in ary_list:        
                        time_3 = temp[2].timestamp()
                        cur_label = False
                        ########處裡前幾筆資料的label
                        if one_hour_pass(time_2, time_3) == 1 and label != 2 and label != 1:
                            #print('start_label')
                            if len(cur_urine_list) and len(cur_weight_list) != 0:
                                cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
                            elif len(cur_urine_list) != 0:
                                cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
                            ########超過一小時 但前幾筆資料沒有6小時尿量紀錄
                            else:
                                cur_label = False
                            #print(cur_label)
                            ########
                            if cur_label != False:
                                if cur_label == 1:
                                    if time_2 - time_1 >= time_in_second_lower:
                                        label = 1
                                        time_4 = time_2
                                    else: 
                                        label = 2
                                else:
                                    label = 0
                                #print(label)

                            ########調整基準時間 計算前後兩筆紀錄是否超過6小時
                            if (time_3 - last_time_3 > 21600) and (label != 1) and (label != 2) and (last_time_3 != 0):
                                if time_2 + 21600 - time_1 >= time_in_second_lower:
                                    label = 1
                                    time_4 = time_2 + 21600
                                else:
                                    label = 2
                                    
                            while time_3 - time_2 >= 3600:
                                time_2 += 3600        
                                    
                            ########清空尿量和體重暫存器
                            cur_weight_list = []
                            cur_urine_list = []
                        ########累積尿量及體重資料
                        if np.isnan(temp[0]) == 0:
                            cur_urine_list.append(temp[0])

                        if np.isnan(temp[1]) == 0 and temp[1] != 0:
                            cur_weight_list.append(temp[1])

                    ########ary_list循環完畢 但urine_list不是空的 => 最後未滿一小時的資料還沒進行label        
                    if len(cur_urine_list) != 0 and label != 2 and label != 1:
                        if len(cur_urine_list) and len(cur_weight_list) != 0:
                            cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
                        elif len(cur_urine_list) != 0:
                            cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
                        ########if anything happens
                        else:
                            cur_label = False
                        ########
                        if cur_label != False:
                            if cur_label == 1:
                                if time_2 - time_1 >= time_in_second_lower:
                                    label = 1
                                    time_4 = time_2
                                else: 
                                    label = 2
                            else:
                                label = 0
                    ########把該append進去處理好的list的資料append進去
                    #print(label)
                    if label == 1:
                        if weight_assumption > 15:
                            patient_list.append(cur_patient)
                            aki += 1
                            end_time_list.append(time_4)
                            if time_1 <= time_4 - time_in_second_upper:
                                start_time_list.append(time_4 - time_in_second_upper)
                            else:
                                #print(cur_patient)
                                start_time_list.append(time_1)
                            label_list.append(label)
                            age_list.append(age)
                            weight_list.append(weight_assumption)
                            base_list.append('u')
                            icu_time_list.append(time_1)
                            leave_time_list.append(time_3)
                    elif label == 0:
                        if weight_assumption > 15:
                            if time_2 - time_1 >= time_in_second_lower:
                                patient_list.append(cur_patient)
                                no_aki += 1
                                end_time_list.append(time_2)
                                if time_1 <= time_2 - time_in_second_upper:
                                    start_time_list.append(time_2 - time_in_second_upper)
                                else:
                                    #print(cur_patient)
                                    start_time_list.append(time_1)
                                label_list.append(label)
                                age_list.append(age)
                                weight_list.append(weight_assumption)
                                base_list.append('u')
                                icu_time_list.append(time_1)
                                leave_time_list.append(time_3)
                    ########結束處理 
            ########更新現今處理病人的病歷號
            last_patient = cur_patient
            cur_patient = int(df[num_of_records][3])
            ########把新的病人append進來
            ary_list = []
            ary_list.append(df[num_of_records])
        ########持續append cur_patient的資料
        else:
            #print('append')
            ary_list.append(df[num_of_records])                        
                        
    ########處理最後一個病人
    age = ary_list[0][4]
    weight_assumption = False
    for temp in ary_list:
        if weight_assumption == False and np.isnan(temp[1]) == 0 and temp[1] != 0:
            weight_assumption = temp[1]
            break
            
    ########確定病人有體重紀錄        
    if weight_assumption != False:
        ########清空變數         
        cur_weight_list = []
        cur_urine_list = []
        time_1 = 0
        time_2 = 0
        time_3 = 0
        last_time_3 = 0
        time_4 = 0
        label = 3

        ########初始化變數
        time_1 = ary_list[0][2].timestamp()
        time_2 = time_1

        ########開始label
        for temp in ary_list:        
            time_3 = temp[2].timestamp()
            ########處裡前幾筆資料的label
            if one_hour_pass(time_2, time_3) == 1 and label != 2 and label != 1:
                if len(cur_urine_list) and len(cur_weight_list) != 0:
                    cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
                elif len(cur_urine_list) != 0:
                    cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
                ########超過一小時 但前幾筆資料沒有6小時尿量紀錄
                else:
                    cur_label = False
                ########
                if cur_label != False:
                    if cur_label == 1:
                        if time_2 - time_1 >= time_in_second_lower:
                            label = 1
                            time_4 = time_2
                        else: 
                            label = 2
                    else:
                        label = 0

                ########調整基準時間 計算前後兩筆紀錄是否超過6小時
                if (time_3 - last_time_3 > 21600) and (label != 1) and (label != 2) and (last_time_3 != 0):
                    if time_2 + 21600 - time_1 >= time_in_second_lower:
                        label = 1
                        time_4 = time_2 + 21600 
                    else:
                        label = 2
                        
                while time_3 - time_2 >= 3600:
                    time_2 += 3600
             
                ########清空尿量和體重暫存器
                cur_weight_list = []
                cur_urine_list = []
            ########累積尿量及體重資料
            if np.isnan(temp[0]) == 0:
                cur_urine_list.append(temp[0])

            if np.isnan(temp[1]) == 0 and temp[1] != 0:
                cur_weight_list.append(temp[1])
                
            ########儲存這次紀錄的時間 之後和下次比對    
            last_time_3 = time_3
        ########ary_list循環完畢 但urine_list不是空的 => 最後未滿一小時的資料還沒進行label        
        if len(cur_urine_list) != 0 and label != 2 and label != 1:
            if len(cur_urine_list) and len(cur_weight_list) != 0:
                cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
            elif len(cur_urine_list) != 0:
                cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
            ########if anything happens
            else:
                cur_label = False
            ########
            if cur_label != False:
                if cur_label == 1:
                    if time_2 - time_1 >= time_in_second_lower:
                        label = 1
                        time_4 = time_2 
                    else: 
                        label = 2
                else:
                    label = 0
    ########單一FILE所有病人處理完畢
    
        ########把該append進去處理好的list的資料append進去
        if label == 1:
            if weight_assumption > 15:
                patient_list.append(cur_patient)
                aki += 1
                end_time_list.append(time_4)
                if time_1 <= time_4 - time_in_second_upper:
                    start_time_list.append(time_4 - time_in_second_upper)
                else:
                    #print(cur_patient)
                    start_time_list.append(time_1)
                label_list.append(label)
                age_list.append(age)
                weight_list.append(weight_assumption)
                base_list.append('u')
                icu_time_list.append(time_1)
                leave_time_list.append(time_3)
        elif label == 0:
            if weight_assumption > 15:
                if time_2 - time_1 >= time_in_second_lower:
                    patient_list.append(cur_patient)
                    no_aki += 1
                    end_time_list.append(time_2)
                    if time_1 <= time_2 - time_in_second_upper:
                        start_time_list.append(time_2 - time_in_second_upper)
                    else:
                        #print(cur_patient)
                        start_time_list.append(time_1)
                    label_list.append(label)
                    age_list.append(age)
                    weight_list.append(weight_assumption)
                    base_list.append('u')
                    icu_time_list.append(time_1)
                    leave_time_list.append(time_3)
            
    ########整理小list    
    patient_list.pop(0)
    label_list.pop(0)
    start_time_list.pop(0)
    end_time_list.pop(0)
    age_list.pop(0)
    weight_list.pop(0)
    base_list.pop(0)
    icu_time_list.pop(0)
    leave_time_list.pop(0)
    
    ########放進大list
    for value in patient_list:
        big_patient_list.append(value)
    for value in label_list:
        big_label_list.append(value)
    for value in start_time_list:
        big_start_time_list.append(value)
    for value in end_time_list:
        big_end_time_list.append(value)
    for value in age_list:
        big_age_list.append(value)
    for value in weight_list:
        big_weight_list.append(value)
    for value in base_list:
        big_base_list.append(value)
    for value in icu_time_list:
        big_icu_time_list.append(value)
    for value in leave_time_list:
        big_leave_time_list.append(value)
        
    #label的結果
    #print(patient_list)        
    #print(label_list)
    #print(len(patient_list))
    #print(len(label_list))
    #print(FILE)
    #print(age_list)
#     break
# print(df)
    
print(aki)
print(no_aki)

SyntaxError: invalid syntax (<ipython-input-12-cc02288b1a47>, line 275)

In [ ]:
big_patient_list = list(map(int, big_patient_list))

In [ ]:
patient_frame = pd.DataFrame(big_patient_list, columns = ['patient_code'])
patient_frame = pd.concat([patient_frame, pd.DataFrame(big_label_list, columns = ['label'])], axis = 1)
patient_frame = pd.concat([patient_frame, pd.DataFrame(big_start_time_list, columns = ['start_time'])], axis = 1)
patient_frame = pd.concat([patient_frame, pd.DataFrame(big_end_time_list, columns = ['end_time'])], axis = 1)
patient_frame = pd.concat([patient_frame, pd.DataFrame(big_age_list, columns = ['age'])], axis = 1)
patient_frame = pd.concat([patient_frame, pd.DataFrame(big_weight_list, columns = ['weight'])], axis = 1)
patient_frame = pd.concat([patient_frame, pd.DataFrame(big_base_list, columns = ['base'])], axis = 1)
patient_frame = pd.concat([patient_frame, pd.DataFrame(big_icu_time_list, columns = ['icu_time'])], axis = 1)
patient_frame = pd.concat([patient_frame, pd.DataFrame(big_leave_time_list, columns = ['leave_time'])], axis = 1)

In [ ]:
if not os.path.exists('../processing_data/label1/' + time_interval_lower_str + '_' + time_interval_upper_str):
    os.mkdir('../processing_data/label1/' + time_interval_lower_str + '_' + time_interval_upper_str)
patient_frame.to_csv('../processing_data/label1/' + time_interval_lower_str + '_' + time_interval_upper_str + '/patient_list.csv', index = False, encoding = 'utf-8')

In [ ]:
print(len(big_age_list))
print(len(big_start_time_list))
print(len(big_patient_list))

### 舊的code

In [ ]:
##########################################################################################
#                  //       //////             //////        //                          #
#                 //      //   //            //   //        //                           #
#                //          //      /////      //         //  //                        #
#               //         //                 //          /////////                      #
#              //        ///////            ////////          //                         #
##########################################################################################
aki = 0
no_aki = 0
month = 1
for FILE in tqdm(urine_filelist):
#     print(month)
    month += 1
    df = pd.read_excel(urine_path + FILE, sheet_name = "匯出資料", usecols = ["6小時總尿量", "尿液輸出前最新體重", "尿液輸出時間", "病歷號", "年齡"]) 
    #6小時總尿量, 體重, 輸出時間, 病歷號 年齡
    df = df.to_numpy()
    patient_list = [0]
    label_list = [0] 
    index = 0
    weight_list = [0]
    cur_weight_list = []
    cur_urine_list = []
    start_time_list = [0]
    end_time_list = [0]
    age_list = [0]
    base_list = [0]
    icu_time_list = [0]
    leave_time_list = [0]
    label = 3 #label: 0 沒有AKI; 1 有AKI, 2 入院23小時內就得AKI; 3 資料有缺失
    time_1 = 0 #進ICU的時間
    time_2 = 0 #一個小時的起始點
    time_3 = 0 #當前紀錄的時間點
    time_4 = 0 #重要的時間點 label 0 不會用到 label 1 等於發病時間前一小時
    last_patient = 0
    cur_patient = 0
    ary_list = []
    weight_assumption = False
    age = False
    
    for num_of_records in range(0,df.shape[0]):
        ########碰到x病人就直接跳過
        if str.isdigit(str(df[num_of_records][3])) == False:
            continue
        ########碰到年齡有問題的病人也跳過
        elif (df[num_of_records][4] < 18 or np.isnan(df[num_of_records][4]) == 1):
            continue
        ########碰到新的病人
        elif int(df[num_of_records][3]) != cur_patient:
            #print(cur_patient)
        ########先處裡舊的 df的第一個病人不會有舊的病人可以處理    
            if cur_patient != last_patient:
                #print('process')
                ########開始處理
                age = ary_list[0][4]
                ########先取得病人最早的體重紀錄 以防有尿量紀錄卻無體重紀錄的狀況
                weight_assumption = False
                #print(ary_list)
                for temp in ary_list:
                    if weight_assumption == False and np.isnan(temp[1]) == 0 and temp[1] != 0:
                        weight_assumption = temp[1]
                        break
                        
                ########確定病人有體重紀錄
                if weight_assumption != False:
                    ########清空變數         
                    cur_weight_list = []
                    cur_urine_list = []
                    time_1 = 0
                    time_2 = 0
                    time_3 = 0
                    last_time_3 = 0
                    time_4 = 0
                    label = 3

                    ########初始化變數
                    time_1 = ary_list[0][2].timestamp()
                    time_2 = time_1
                    ########開始label
                    #print('start_label')
                    for temp in ary_list:        
                        time_3 = temp[2].timestamp()
                        cur_label = False
                        ########處裡前幾筆資料的label
                        if one_hour_pass(time_2, time_3) == 1 and label != 2 and label != 1:
                            #print('start_label')
                            if len(cur_urine_list) and len(cur_weight_list) != 0:
                                cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
                            elif len(cur_urine_list) != 0:
                                cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
                            ########超過一小時 但前幾筆資料沒有6小時尿量紀錄
                            else:
                                cur_label = False
                            #print(cur_label)
                            ########
                            if cur_label != False:
                                if cur_label == 1:
                                    if time_2 - time_1 >= 43200:
                                        label = 1
                                        time_4 = time_2 - 3600
                                    else: 
                                        label = 2
                                else:
                                    label = 0
                                #print(label)

                            ########調整基準時間 計算前後兩筆紀錄是否超過6小時
                            if (time_3 - last_time_3 > 21600) and (label != 1) and (label != 2) and (last_time_3 != 0):
                                if time_2 + 21600 - time_1 >= 43200:
                                    label = 1
                                    time_4 = time_2 + 21600 - 3600
                                else:
                                    label = 2
                                    
                            while time_3 - time_2 >= 3600:
                                time_2 += 3600        
                                    
                            ########清空尿量和體重暫存器
                            cur_weight_list = []
                            cur_urine_list = []
                        ########累積尿量及體重資料
                        if np.isnan(temp[0]) == 0:
                            cur_urine_list.append(temp[0])

                        if np.isnan(temp[1]) == 0 and temp[1] != 0:
                            cur_weight_list.append(temp[1])

                    ########ary_list循環完畢 但urine_list不是空的 => 最後未滿一小時的資料還沒進行label        
                    if len(cur_urine_list) != 0 and label != 2 and label != 1:
                        if len(cur_urine_list) and len(cur_weight_list) != 0:
                            cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
                        elif len(cur_urine_list) != 0:
                            cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
                        ########if anything happens
                        else:
                            cur_label = False
                        ########
                        if cur_label != False:
                            if cur_label == 1:
                                ######## 86400為24小時
                                # if time_2 - time_1 >= 86400:
                                #     label = 1
                                #     time_4 = time_2 - 3600
                                ######## 43200為12小時
                                if time_2 - time_1 >= 43200:
                                    label = 1
                                    time_4 = time_2 - 3600
                                else: 
                                    label = 2
                            else:
                                label = 0
                    ########把該append進去處理好的list的資料append進去
                    #print(label)
                    if label == 1:
                        if weight_assumption > 15:
                            patient_list.append(cur_patient)
                            aki += 1
                            end_time_list.append(time_4)
                            if time_1 <= time_4 - 82800:
                                start_time_list.append(time_4 - 82800)
                            else:
                                #print(cur_patient)
                                start_time_list.append(time_1)
                            label_list.append(label)
                            age_list.append(age)
                            weight_list.append(weight_assumption)
                            base_list.append('u')
                            icu_time_list.append(time_1)
                            leave_time_list.append(time_3)
                    elif label == 0:
                        if weight_assumption > 15:
                            if time_2 - time_1 >= 43200:
                                patient_list.append(cur_patient)
                                no_aki += 1
                                end_time_list.append(time_2 - 3600)
                                if time_1 <= time_2 - 86400:
                                    start_time_list.append(time_2 - 86400)
                                else:
                                    #print(cur_patient)
                                    start_time_list.append(time_1)
                                label_list.append(label)
                                age_list.append(age)
                                weight_list.append(weight_assumption)
                                base_list.append('u')
                                icu_time_list.append(time_1)
                                leave_time_list.append(time_3)
                    ########結束處理 
            ########更新現今處理病人的病歷號
            last_patient = cur_patient
            cur_patient = int(df[num_of_records][3])
            ########把新的病人append進來
            ary_list = []
            ary_list.append(df[num_of_records])
        ########持續append cur_patient的資料
        else:
            #print('append')
            ary_list.append(df[num_of_records])                        
                        
    ########處理最後一個病人
    age = ary_list[0][4]
    weight_assumption = False
    for temp in ary_list:
        if weight_assumption == False and np.isnan(temp[1]) == 0 and temp[1] != 0:
            weight_assumption = temp[1]
            break
            
    ########確定病人有體重紀錄        
    if weight_assumption != False:
        ########清空變數         
        cur_weight_list = []
        cur_urine_list = []
        time_1 = 0
        time_2 = 0
        time_3 = 0
        last_time_3 = 0
        time_4 = 0
        label = 3

        ########初始化變數
        time_1 = ary_list[0][2].timestamp()
        time_2 = time_1

        ########開始label
        for temp in ary_list:        
            time_3 = temp[2].timestamp()
            ########處裡前幾筆資料的label
            if one_hour_pass(time_2, time_3) == 1 and label != 2 and label != 1:
                if len(cur_urine_list) and len(cur_weight_list) != 0:
                    cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
                elif len(cur_urine_list) != 0:
                    cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
                ########超過一小時 但前幾筆資料沒有6小時尿量紀錄
                else:
                    cur_label = False
                ########
                if cur_label != False:
                    if cur_label == 1:
                        if time_2 - time_1 >= 43200:
                            label = 1
                            time_4 = time_2 - 3600
                        else: 
                            label = 2
                    else:
                        label = 0

                ########調整基準時間 計算前後兩筆紀錄是否超過6小時
                if (time_3 - last_time_3 > 21600) and (label != 1) and (label != 2) and (last_time_3 != 0):
                    if time_2 + 21600 - time_1 >= 43200:
                        label = 1
                        time_4 = time_2 + 21600 - 3600
                    else:
                        label = 2
                        
                while time_3 - time_2 >= 3600:
                    time_2 += 3600
             
                ########清空尿量和體重暫存器
                cur_weight_list = []
                cur_urine_list = []
            ########累積尿量及體重資料
            if np.isnan(temp[0]) == 0:
                cur_urine_list.append(temp[0])

            if np.isnan(temp[1]) == 0 and temp[1] != 0:
                cur_weight_list.append(temp[1])
                
            ########儲存這次紀錄的時間 之後和下次比對    
            last_time_3 = time_3
        ########ary_list循環完畢 但urine_list不是空的 => 最後未滿一小時的資料還沒進行label        
        if len(cur_urine_list) != 0 and label != 2 and label != 1:
            if len(cur_urine_list) and len(cur_weight_list) != 0:
                cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
            elif len(cur_urine_list) != 0:
                cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
            ########if anything happens
            else:
                cur_label = False
            ########
            if cur_label != False:
                if cur_label == 1:
                    if time_2 - time_1 >= 43200:
                        label = 1
                        time_4 = time_2 - 3600
                    else: 
                        label = 2
                else:
                    label = 0
    ########單一FILE所有病人處理完畢
    
        ########把該append進去處理好的list的資料append進去
        if label == 1:
            if weight_assumption > 15:
                patient_list.append(cur_patient)
                aki += 1
                end_time_list.append(time_4)
                if time_1 <= time_4 - 82800:
                    start_time_list.append(time_4 - 82800)
                else:
                    #print(cur_patient)
                    start_time_list.append(time_1)
                label_list.append(label)
                age_list.append(age)
                weight_list.append(weight_assumption)
                base_list.append('u')
                icu_time_list.append(time_1)
                leave_time_list.append(time_3)
        elif label == 0:
            if weight_assumption > 15:
                if time_2 - time_1 >= 43200:
                    patient_list.append(cur_patient)
                    no_aki += 1
                    end_time_list.append(time_2 - 3600)
                    if time_1 <= time_2 - 86400:
                        start_time_list.append(time_2 - 86400)
                    else:
                        #print(cur_patient)
                        start_time_list.append(time_1)
                    label_list.append(label)
                    age_list.append(age)
                    weight_list.append(weight_assumption)
                    base_list.append('u')
                    icu_time_list.append(time_1)
                    leave_time_list.append(time_3)
            
    ########整理小list    
    patient_list.pop(0)
    label_list.pop(0)
    start_time_list.pop(0)
    end_time_list.pop(0)
    age_list.pop(0)
    weight_list.pop(0)
    base_list.pop(0)
    icu_time_list.pop(0)
    leave_time_list.pop(0)
    
    ########放進大list
    for value in patient_list:
        big_patient_list.append(value)
    for value in label_list:
        big_label_list.append(value)
    for value in start_time_list:
        big_start_time_list.append(value)
    for value in end_time_list:
        big_end_time_list.append(value)
    for value in age_list:
        big_age_list.append(value)
    for value in weight_list:
        big_weight_list.append(value)
    for value in base_list:
        big_base_list.append(value)
    for value in icu_time_list:
        big_icu_time_list.append(value)
    for value in leave_time_list:
        big_leave_time_list.append(value)
        
    #label的結果
    #print(patient_list)        
    #print(label_list)
    #print(len(patient_list))
    #print(len(label_list))
    #print(FILE)
    #print(age_list)
#     break
# print(df)
    
print(aki)
print(no_aki)

In [ ]:
##########################################################################################
#                   //                             //      //////                        #
#                 //                              //     //   //                         #
#                //////              /////       //         //                           #
#               //    //                        //        //                             #
#               ///////                        //       ////////                         #
##########################################################################################
aki = 0
no_aki = 0
month = 1
for FILE in tqdm(urine_filelist):
#     print(month)
    month += 1
    df = pd.read_excel(urine_path + FILE, sheet_name = "匯出資料", usecols = ["6小時總尿量", "尿液輸出前最新體重", "尿液輸出時間", "病歷號", "年齡"]) 
    #6小時總尿量, 體重, 輸出時間, 病歷號 年齡
    df = df.to_numpy()
    patient_list = [0]
    label_list = [0] 
    index = 0
    weight_list = [0]
    cur_weight_list = []
    cur_urine_list = []
    start_time_list = [0]
    end_time_list = [0]
    age_list = [0]
    base_list = [0]
    icu_time_list = [0]
    leave_time_list = [0]
    label = 3 #label: 0 沒有AKI; 1 有AKI, 2 入院23小時內就得AKI; 3 資料有缺失
    time_1 = 0 #進ICU的時間
    time_2 = 0 #一個小時的起始點
    time_3 = 0 #當前紀錄的時間點
    time_4 = 0 #重要的時間點 label 0 不會用到 label 1 等於發病時間前一小時
    last_patient = 0
    cur_patient = 0
    ary_list = []
    weight_assumption = False
    age = False
    
    for num_of_records in range(0,df.shape[0]):
        ########碰到x病人就直接跳過
        if str.isdigit(str(df[num_of_records][3])) == False:
            continue
        ########碰到年齡有問題的病人也跳過
        elif (df[num_of_records][4] < 18 or np.isnan(df[num_of_records][4]) == 1):
            continue
        ########碰到新的病人
        elif int(df[num_of_records][3]) != cur_patient:
            #print(cur_patient)
        ########先處裡舊的 df的第一個病人不會有舊的病人可以處理    
            if cur_patient != last_patient:
                #print('process')
                ########開始處理
                age = ary_list[0][4]
                ########先取得病人最早的體重紀錄 以防有尿量紀錄卻無體重紀錄的狀況
                weight_assumption = False
                #print(ary_list)
                for temp in ary_list:
                    if weight_assumption == False and np.isnan(temp[1]) == 0 and temp[1] != 0:
                        weight_assumption = temp[1]
                        break
                        
                ########確定病人有體重紀錄
                if weight_assumption != False:
                    ########清空變數         
                    cur_weight_list = []
                    cur_urine_list = []
                    time_1 = 0
                    time_2 = 0
                    time_3 = 0
                    last_time_3 = 0
                    time_4 = 0
                    label = 3

                    ########初始化變數
                    time_1 = ary_list[0][2].timestamp()
                    time_2 = time_1
                    ########開始label
                    #print('start_label')
                    for temp in ary_list:        
                        time_3 = temp[2].timestamp()
                        cur_label = False
                        ########處裡前幾筆資料的label
                        if one_hour_pass(time_2, time_3) == 1 and label != 2 and label != 1:
                            #print('start_label')
                            if len(cur_urine_list) and len(cur_weight_list) != 0:
                                cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
                            elif len(cur_urine_list) != 0:
                                cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
                            ########超過一小時 但前幾筆資料沒有6小時尿量紀錄
                            else:
                                cur_label = False
                            #print(cur_label)
                            ########
                            if cur_label != False:
                                if cur_label == 1:
                                    if time_2 - time_1 >= 21600:
                                        label = 1
                                        time_4 = time_2 - 3600
                                    else: 
                                        label = 2
                                else:
                                    label = 0
                                #print(label)

                            ########調整基準時間 計算前後兩筆紀錄是否超過6小時
                            if (time_3 - last_time_3 > 21600) and (label != 1) and (label != 2) and (last_time_3 != 0):
                                if time_2 + 21600 - time_1 >= 21600:
                                    label = 1
                                    time_4 = time_2 + 21600 - 3600
                                else:
                                    label = 2
                                    
                            while time_3 - time_2 >= 3600:
                                time_2 += 3600        
                                    
                            ########清空尿量和體重暫存器
                            cur_weight_list = []
                            cur_urine_list = []
                        ########累積尿量及體重資料
                        if np.isnan(temp[0]) == 0:
                            cur_urine_list.append(temp[0])

                        if np.isnan(temp[1]) == 0 and temp[1] != 0:
                            cur_weight_list.append(temp[1])

                    ########ary_list循環完畢 但urine_list不是空的 => 最後未滿一小時的資料還沒進行label        
                    if len(cur_urine_list) != 0 and label != 2 and label != 1:
                        if len(cur_urine_list) and len(cur_weight_list) != 0:
                            cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
                        elif len(cur_urine_list) != 0:
                            cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
                        ########if anything happens
                        else:
                            cur_label = False
                        ########
                        if cur_label != False:
                            if cur_label == 1:
                                ######## 86400為24小時
                                # if time_2 - time_1 >= 86400:
                                #     label = 1
                                #     time_4 = time_2 - 3600
                                ######## 43200為12小時
                                if time_2 - time_1 >= 21600:
                                    label = 1
                                    time_4 = time_2 - 3600
                                else: 
                                    label = 2
                            else:
                                label = 0
                    ########把該append進去處理好的list的資料append進去
                    #print(label)
                    if label == 1:
                        if weight_assumption > 15:
                            patient_list.append(cur_patient)
                            aki += 1
                            end_time_list.append(time_4)
                            if time_1 <= time_4 - 39600:
                                start_time_list.append(time_4 - 39600)
                            else:
                                #print(cur_patient)
                                start_time_list.append(time_1)
                            label_list.append(label)
                            age_list.append(age)
                            weight_list.append(weight_assumption)
                            base_list.append('u')
                            icu_time_list.append(time_1)
                            leave_time_list.append(time_3)
                    elif label == 0:
                        if weight_assumption > 15:
                            if time_2 - time_1 >= 21600:
                                patient_list.append(cur_patient)
                                no_aki += 1
                                end_time_list.append(time_2 - 3600)
                                if time_1 <= time_2 - 43200:
                                    start_time_list.append(time_2 - 43200)
                                else:
                                    #print(cur_patient)
                                    start_time_list.append(time_1)
                                label_list.append(label)
                                age_list.append(age)
                                weight_list.append(weight_assumption)
                                base_list.append('u')
                                icu_time_list.append(time_1)
                                leave_time_list.append(time_3)
                    ########結束處理 
            ########更新現今處理病人的病歷號
            last_patient = cur_patient
            cur_patient = int(df[num_of_records][3])
            ########把新的病人append進來
            ary_list = []
            ary_list.append(df[num_of_records])
        ########持續append cur_patient的資料
        else:
            #print('append')
            ary_list.append(df[num_of_records])                        
                        
    ########處理最後一個病人
    age = ary_list[0][4]
    weight_assumption = False
    for temp in ary_list:
        if weight_assumption == False and np.isnan(temp[1]) == 0 and temp[1] != 0:
            weight_assumption = temp[1]
            break
            
    ########確定病人有體重紀錄        
    if weight_assumption != False:
        ########清空變數         
        cur_weight_list = []
        cur_urine_list = []
        time_1 = 0
        time_2 = 0
        time_3 = 0
        last_time_3 = 0
        time_4 = 0
        label = 3

        ########初始化變數
        time_1 = ary_list[0][2].timestamp()
        time_2 = time_1

        ########開始label
        for temp in ary_list:        
            time_3 = temp[2].timestamp()
            ########處裡前幾筆資料的label
            if one_hour_pass(time_2, time_3) == 1 and label != 2 and label != 1:
                if len(cur_urine_list) and len(cur_weight_list) != 0:
                    cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
                elif len(cur_urine_list) != 0:
                    cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
                ########超過一小時 但前幾筆資料沒有6小時尿量紀錄
                else:
                    cur_label = False
                ########
                if cur_label != False:
                    if cur_label == 1:
                        if time_2 - time_1 >= 21600:
                            label = 1
                            time_4 = time_2 - 3600
                        else: 
                            label = 2
                    else:
                        label = 0

                ########調整基準時間 計算前後兩筆紀錄是否超過6小時
                if (time_3 - last_time_3 > 21600) and (label != 1) and (label != 2) and (last_time_3 != 0):
                    if time_2 + 21600 - time_1 >= 21600:
                        label = 1
                        time_4 = time_2 + 21600 - 3600
                    else:
                        label = 2
                        
                while time_3 - time_2 >= 3600:
                    time_2 += 3600
             
                ########清空尿量和體重暫存器
                cur_weight_list = []
                cur_urine_list = []
            ########累積尿量及體重資料
            if np.isnan(temp[0]) == 0:
                cur_urine_list.append(temp[0])

            if np.isnan(temp[1]) == 0 and temp[1] != 0:
                cur_weight_list.append(temp[1])
                
            ########儲存這次紀錄的時間 之後和下次比對    
            last_time_3 = time_3
        ########ary_list循環完畢 但urine_list不是空的 => 最後未滿一小時的資料還沒進行label        
        if len(cur_urine_list) != 0 and label != 2 and label != 1:
            if len(cur_urine_list) and len(cur_weight_list) != 0:
                cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
            elif len(cur_urine_list) != 0:
                cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
            ########if anything happens
            else:
                cur_label = False
            ########
            if cur_label != False:
                if cur_label == 1:
                    if time_2 - time_1 >= 21600:
                        label = 1
                        time_4 = time_2 - 3600
                    else: 
                        label = 2
                else:
                    label = 0
    ########單一FILE所有病人處理完畢
    
        ########把該append進去處理好的list的資料append進去
        if label == 1:
            if weight_assumption > 15:
                patient_list.append(cur_patient)
                aki += 1
                end_time_list.append(time_4)
                if time_1 <= time_4 - 39600:
                    start_time_list.append(time_4 - 39600)
                else:
                    #print(cur_patient)
                    start_time_list.append(time_1)
                label_list.append(label)
                age_list.append(age)
                weight_list.append(weight_assumption)
                base_list.append('u')
                icu_time_list.append(time_1)
                leave_time_list.append(time_3)
        elif label == 0:
            if weight_assumption > 15:
                if time_2 - time_1 >= 21600:
                    patient_list.append(cur_patient)
                    no_aki += 1
                    end_time_list.append(time_2 - 3600)
                    if time_1 <= time_2 - 43200:
                        start_time_list.append(time_2 - 43200)
                    else:
                        #print(cur_patient)
                        start_time_list.append(time_1)
                    label_list.append(label)
                    age_list.append(age)
                    weight_list.append(weight_assumption)
                    base_list.append('u')
                    icu_time_list.append(time_1)
                    leave_time_list.append(time_3)
            
    ########整理小list    
    patient_list.pop(0)
    label_list.pop(0)
    start_time_list.pop(0)
    end_time_list.pop(0)
    age_list.pop(0)
    weight_list.pop(0)
    base_list.pop(0)
    icu_time_list.pop(0)
    leave_time_list.pop(0)
    
    ########放進大list
    for value in patient_list:
        big_patient_list.append(value)
    for value in label_list:
        big_label_list.append(value)
    for value in start_time_list:
        big_start_time_list.append(value)
    for value in end_time_list:
        big_end_time_list.append(value)
    for value in age_list:
        big_age_list.append(value)
    for value in weight_list:
        big_weight_list.append(value)
    for value in base_list:
        big_base_list.append(value)
    for value in icu_time_list:
        big_icu_time_list.append(value)
    for value in leave_time_list:
        big_leave_time_list.append(value)
        
    #label的結果
    #print(patient_list)        
    #print(label_list)
    #print(len(patient_list))
    #print(len(label_list))
    #print(FILE)
    #print(age_list)
#     break
# print(df)
    
print(aki)
print(no_aki)

In [ ]:
##########################################################################################
#                   //                         //////        //                          #
#                 //                         //   //        //                           #
#                //////              /////      //         //  //                        #
#               //    //                      //          /////////                      #
#               ///////                     ////////          //                         #
##########################################################################################
aki = 0
no_aki = 0
month = 1
for FILE in tqdm(urine_filelist):
#     print(month)
    month += 1
    df = pd.read_excel(urine_path + FILE, sheet_name = "匯出資料", usecols = ["6小時總尿量", "尿液輸出前最新體重", "尿液輸出時間", "病歷號", "年齡"]) 
    #6小時總尿量, 體重, 輸出時間, 病歷號 年齡
    df = df.to_numpy()
    patient_list = [0]
    label_list = [0] 
    index = 0
    weight_list = [0]
    cur_weight_list = []
    cur_urine_list = []
    start_time_list = [0]
    end_time_list = [0]
    age_list = [0]
    base_list = [0]
    icu_time_list = [0]
    leave_time_list = [0]
    label = 3 #label: 0 沒有AKI; 1 有AKI, 2 入院23小時內就得AKI; 3 資料有缺失
    time_1 = 0 #進ICU的時間
    time_2 = 0 #一個小時的起始點
    time_3 = 0 #當前紀錄的時間點
    time_4 = 0 #重要的時間點 label 0 不會用到 label 1 等於發病時間前一小時
    last_patient = 0
    cur_patient = 0
    ary_list = []
    weight_assumption = False
    age = False
    
    for num_of_records in range(0,df.shape[0]):
        ########碰到x病人就直接跳過
        if str.isdigit(str(df[num_of_records][3])) == False:
            continue
        ########碰到年齡有問題的病人也跳過
        elif (df[num_of_records][4] < 18 or np.isnan(df[num_of_records][4]) == 1):
            continue
        ########碰到新的病人
        elif int(df[num_of_records][3]) != cur_patient:
            #print(cur_patient)
        ########先處裡舊的 df的第一個病人不會有舊的病人可以處理    
            if cur_patient != last_patient:
                #print('process')
                ########開始處理
                age = ary_list[0][4]
                ########先取得病人最早的體重紀錄 以防有尿量紀錄卻無體重紀錄的狀況
                weight_assumption = False
                #print(ary_list)
                for temp in ary_list:
                    if weight_assumption == False and np.isnan(temp[1]) == 0 and temp[1] != 0:
                        weight_assumption = temp[1]
                        break
                        
                ########確定病人有體重紀錄
                if weight_assumption != False:
                    ########清空變數         
                    cur_weight_list = []
                    cur_urine_list = []
                    time_1 = 0
                    time_2 = 0
                    time_3 = 0
                    last_time_3 = 0
                    time_4 = 0
                    label = 3

                    ########初始化變數
                    time_1 = ary_list[0][2].timestamp()
                    time_2 = time_1
                    ########開始label
                    #print('start_label')
                    for temp in ary_list:        
                        time_3 = temp[2].timestamp()
                        cur_label = False
                        ########處裡前幾筆資料的label
                        if one_hour_pass(time_2, time_3) == 1 and label != 2 and label != 1:
                            #print('start_label')
                            if len(cur_urine_list) and len(cur_weight_list) != 0:
                                cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
                            elif len(cur_urine_list) != 0:
                                cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
                            ########超過一小時 但前幾筆資料沒有6小時尿量紀錄
                            else:
                                cur_label = False
                            #print(cur_label)
                            ########
                            if cur_label != False:
                                if cur_label == 1:
                                    if time_2 - time_1 >= 21600:
                                        label = 1
                                        time_4 = time_2 - 3600
                                    else: 
                                        label = 2
                                else:
                                    label = 0
                                #print(label)

                            ########調整基準時間 計算前後兩筆紀錄是否超過6小時
                            if (time_3 - last_time_3 > 21600) and (label != 1) and (label != 2) and (last_time_3 != 0):
                                if time_2 + 21600 - time_1 >= 21600:
                                    label = 1
                                    time_4 = time_2 + 21600 - 3600
                                else:
                                    label = 2
                                    
                            while time_3 - time_2 >= 3600:
                                time_2 += 3600        
                                    
                            ########清空尿量和體重暫存器
                            cur_weight_list = []
                            cur_urine_list = []
                        ########累積尿量及體重資料
                        if np.isnan(temp[0]) == 0:
                            cur_urine_list.append(temp[0])

                        if np.isnan(temp[1]) == 0 and temp[1] != 0:
                            cur_weight_list.append(temp[1])

                    ########ary_list循環完畢 但urine_list不是空的 => 最後未滿一小時的資料還沒進行label        
                    if len(cur_urine_list) != 0 and label != 2 and label != 1:
                        if len(cur_urine_list) and len(cur_weight_list) != 0:
                            cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
                        elif len(cur_urine_list) != 0:
                            cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
                        ########if anything happens
                        else:
                            cur_label = False
                        ########
                        if cur_label != False:
                            if cur_label == 1:
                                ######## 86400為24小時
                                # if time_2 - time_1 >= 86400:
                                #     label = 1
                                #     time_4 = time_2 - 3600
                                ######## 43200為12小時
                                if time_2 - time_1 >= 21600:
                                    label = 1
                                    time_4 = time_2 - 3600
                                else: 
                                    label = 2
                            else:
                                label = 0
                    ########把該append進去處理好的list的資料append進去
                    #print(label)
                    if label == 1:
                        if weight_assumption > 15:
                            patient_list.append(cur_patient)
                            aki += 1
                            end_time_list.append(time_4)
                            if time_1 <= time_4 - 82800:
                                start_time_list.append(time_4 - 82800)
                            else:
                                #print(cur_patient)
                                start_time_list.append(time_1)
                            label_list.append(label)
                            age_list.append(age)
                            weight_list.append(weight_assumption)
                            base_list.append('u')
                            icu_time_list.append(time_1)
                            leave_time_list.append(time_3)
                    elif label == 0:
                        if weight_assumption > 15:
                            if time_2 - time_1 >= 21600:
                                patient_list.append(cur_patient)
                                no_aki += 1
                                end_time_list.append(time_2 - 3600)
                                if time_1 <= time_2 - 86400:
                                    start_time_list.append(time_2 - 86400)
                                else:
                                    #print(cur_patient)
                                    start_time_list.append(time_1)
                                label_list.append(label)
                                age_list.append(age)
                                weight_list.append(weight_assumption)
                                base_list.append('u')
                                icu_time_list.append(time_1)
                                leave_time_list.append(time_3)
                    ########結束處理 
            ########更新現今處理病人的病歷號
            last_patient = cur_patient
            cur_patient = int(df[num_of_records][3])
            ########把新的病人append進來
            ary_list = []
            ary_list.append(df[num_of_records])
        ########持續append cur_patient的資料
        else:
            #print('append')
            ary_list.append(df[num_of_records])                        
                        
    ########處理最後一個病人
    age = ary_list[0][4]
    weight_assumption = False
    for temp in ary_list:
        if weight_assumption == False and np.isnan(temp[1]) == 0 and temp[1] != 0:
            weight_assumption = temp[1]
            break
            
    ########確定病人有體重紀錄        
    if weight_assumption != False:
        ########清空變數         
        cur_weight_list = []
        cur_urine_list = []
        time_1 = 0
        time_2 = 0
        time_3 = 0
        last_time_3 = 0
        time_4 = 0
        label = 3

        ########初始化變數
        time_1 = ary_list[0][2].timestamp()
        time_2 = time_1

        ########開始label
        for temp in ary_list:        
            time_3 = temp[2].timestamp()
            ########處裡前幾筆資料的label
            if one_hour_pass(time_2, time_3) == 1 and label != 2 and label != 1:
                if len(cur_urine_list) and len(cur_weight_list) != 0:
                    cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
                elif len(cur_urine_list) != 0:
                    cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
                ########超過一小時 但前幾筆資料沒有6小時尿量紀錄
                else:
                    cur_label = False
                ########
                if cur_label != False:
                    if cur_label == 1:
                        if time_2 - time_1 >= 21600:
                            label = 1
                            time_4 = time_2 - 3600
                        else: 
                            label = 2
                    else:
                        label = 0

                ########調整基準時間 計算前後兩筆紀錄是否超過6小時
                if (time_3 - last_time_3 > 21600) and (label != 1) and (label != 2) and (last_time_3 != 0):
                    if time_2 + 21600 - time_1 >= 21600:
                        label = 1
                        time_4 = time_2 + 21600 - 3600
                    else:
                        label = 2
                        
                while time_3 - time_2 >= 3600:
                    time_2 += 3600
             
                ########清空尿量和體重暫存器
                cur_weight_list = []
                cur_urine_list = []
            ########累積尿量及體重資料
            if np.isnan(temp[0]) == 0:
                cur_urine_list.append(temp[0])

            if np.isnan(temp[1]) == 0 and temp[1] != 0:
                cur_weight_list.append(temp[1])
                
            ########儲存這次紀錄的時間 之後和下次比對    
            last_time_3 = time_3
        ########ary_list循環完畢 但urine_list不是空的 => 最後未滿一小時的資料還沒進行label        
        if len(cur_urine_list) != 0 and label != 2 and label != 1:
            if len(cur_urine_list) and len(cur_weight_list) != 0:
                cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
            elif len(cur_urine_list) != 0:
                cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
            ########if anything happens
            else:
                cur_label = False
            ########
            if cur_label != False:
                if cur_label == 1:
                    if time_2 - time_1 >= 21600:
                        label = 1
                        time_4 = time_2 - 3600
                    else: 
                        label = 2
                else:
                    label = 0
    ########單一FILE所有病人處理完畢
    
        ########把該append進去處理好的list的資料append進去
        if label == 1:
            if weight_assumption > 15:
                patient_list.append(cur_patient)
                aki += 1
                end_time_list.append(time_4)
                if time_1 <= time_4 - 82800:
                    start_time_list.append(time_4 - 82800)
                else:
                    #print(cur_patient)
                    start_time_list.append(time_1)
                label_list.append(label)
                age_list.append(age)
                weight_list.append(weight_assumption)
                base_list.append('u')
                icu_time_list.append(time_1)
                leave_time_list.append(time_3)
        elif label == 0:
            if weight_assumption > 15:
                if time_2 - time_1 >= 21600:
                    patient_list.append(cur_patient)
                    no_aki += 1
                    end_time_list.append(time_2 - 3600)
                    if time_1 <= time_2 - 86400:
                        start_time_list.append(time_2 - 86400)
                    else:
                        #print(cur_patient)
                        start_time_list.append(time_1)
                    label_list.append(label)
                    age_list.append(age)
                    weight_list.append(weight_assumption)
                    base_list.append('u')
                    icu_time_list.append(time_1)
                    leave_time_list.append(time_3)
            
    ########整理小list    
    patient_list.pop(0)
    label_list.pop(0)
    start_time_list.pop(0)
    end_time_list.pop(0)
    age_list.pop(0)
    weight_list.pop(0)
    base_list.pop(0)
    icu_time_list.pop(0)
    leave_time_list.pop(0)
    
    ########放進大list
    for value in patient_list:
        big_patient_list.append(value)
    for value in label_list:
        big_label_list.append(value)
    for value in start_time_list:
        big_start_time_list.append(value)
    for value in end_time_list:
        big_end_time_list.append(value)
    for value in age_list:
        big_age_list.append(value)
    for value in weight_list:
        big_weight_list.append(value)
    for value in base_list:
        big_base_list.append(value)
    for value in icu_time_list:
        big_icu_time_list.append(value)
    for value in leave_time_list:
        big_leave_time_list.append(value)
        
    #label的結果
    #print(patient_list)        
    #print(label_list)
    #print(len(patient_list))
    #print(len(label_list))
    #print(FILE)
    #print(age_list)
#     break
# print(df)
    
print(aki)
print(no_aki)

In [ ]:
##########################################################################################
#                         //             //////                                          #
#                        //            //   //                                           #
#                       //                //                                             #
#                      //               //                                               #
#                     //              ////////                                           #
##########################################################################################
aki = 0
no_aki = 0
month = 1
for FILE in tqdm(urine_filelist):
#     print(month)
    month += 1
    df = pd.read_excel(urine_path + FILE, sheet_name = "匯出資料", usecols = ["6小時總尿量", "尿液輸出前最新體重", "尿液輸出時間", "病歷號", "年齡"]) 
    #6小時總尿量, 體重, 輸出時間, 病歷號 年齡
    df = df.to_numpy()
    patient_list = [0]
    label_list = [0] 
    index = 0
    weight_list = [0]
    cur_weight_list = []
    cur_urine_list = []
    start_time_list = [0]
    end_time_list = [0]
    age_list = [0]
    base_list = [0]
    icu_time_list = [0]
    leave_time_list = [0]
    label = 3 #label: 0 沒有AKI; 1 有AKI, 2 入院23小時內就得AKI; 3 資料有缺失
    time_1 = 0 #進ICU的時間
    time_2 = 0 #一個小時的起始點
    time_3 = 0 #當前紀錄的時間點
    time_4 = 0 #重要的時間點 label 0 不會用到 label 1 等於發病時間前一小時
    last_patient = 0
    cur_patient = 0
    ary_list = []
    weight_assumption = False
    age = False
    
    for num_of_records in range(0,df.shape[0]):
        ########碰到x病人就直接跳過
        if str.isdigit(str(df[num_of_records][3])) == False:
            continue
        ########碰到年齡有問題的病人也跳過
        elif (df[num_of_records][4] < 18 or np.isnan(df[num_of_records][4]) == 1):
            continue
        ########碰到新的病人
        elif int(df[num_of_records][3]) != cur_patient:
            #print(cur_patient)
        ########先處裡舊的 df的第一個病人不會有舊的病人可以處理    
            if cur_patient != last_patient:
                #print('process')
                ########開始處理
                age = ary_list[0][4]
                ########先取得病人最早的體重紀錄 以防有尿量紀錄卻無體重紀錄的狀況
                weight_assumption = False
                #print(ary_list)
                for temp in ary_list:
                    if weight_assumption == False and np.isnan(temp[1]) == 0 and temp[1] != 0:
                        weight_assumption = temp[1]
                        break
                        
                ########確定病人有體重紀錄
                if weight_assumption != False:
                    ########清空變數         
                    cur_weight_list = []
                    cur_urine_list = []
                    time_1 = 0
                    time_2 = 0
                    time_3 = 0
                    last_time_3 = 0
                    time_4 = 0
                    label = 3

                    ########初始化變數
                    time_1 = ary_list[0][2].timestamp()
                    time_2 = time_1
                    ########開始label
                    #print('start_label')
                    for temp in ary_list:        
                        time_3 = temp[2].timestamp()
                        cur_label = False
                        ########處裡前幾筆資料的label
                        if one_hour_pass(time_2, time_3) == 1 and label != 2 and label != 1:
                            #print('start_label')
                            if len(cur_urine_list) and len(cur_weight_list) != 0:
                                cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
                            elif len(cur_urine_list) != 0:
                                cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
                            ########超過一小時 但前幾筆資料沒有6小時尿量紀錄
                            else:
                                cur_label = False
                            #print(cur_label)
                            ########
                            if cur_label != False:
                                if cur_label == 1:
                                    if time_2 - time_1 >= 43200:
                                        label = 1
                                        time_4 = time_2 - 3600
                                    else: 
                                        label = 2
                                else:
                                    label = 0
                                #print(label)

                            ########調整基準時間 計算前後兩筆紀錄是否超過6小時
                            if (time_3 - last_time_3 > 21600) and (label != 1) and (label != 2) and (last_time_3 != 0):
                                if time_2 + 21600 - time_1 >= 43200:
                                    label = 1
                                    time_4 = time_2 + 21600 - 3600
                                else:
                                    label = 2
                                    
                            while time_3 - time_2 >= 3600:
                                time_2 += 3600        
                                    
                            ########清空尿量和體重暫存器
                            cur_weight_list = []
                            cur_urine_list = []
                        ########累積尿量及體重資料
                        if np.isnan(temp[0]) == 0:
                            cur_urine_list.append(temp[0])

                        if np.isnan(temp[1]) == 0 and temp[1] != 0:
                            cur_weight_list.append(temp[1])

                    ########ary_list循環完畢 但urine_list不是空的 => 最後未滿一小時的資料還沒進行label        
                    if len(cur_urine_list) != 0 and label != 2 and label != 1:
                        if len(cur_urine_list) and len(cur_weight_list) != 0:
                            cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
                        elif len(cur_urine_list) != 0:
                            cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
                        ########if anything happens
                        else:
                            cur_label = False
                        ########
                        if cur_label != False:
                            if cur_label == 1:
                                ######## 86400為24小時
                                # if time_2 - time_1 >= 86400:
                                #     label = 1
                                #     time_4 = time_2 - 3600
                                ######## 43200為12小時
                                if time_2 - time_1 >= 43200:
                                    label = 1
                                    time_4 = time_2 - 3600
                                else: 
                                    label = 2
                            else:
                                label = 0
                    ########把該append進去處理好的list的資料append進去
                    #print(label)
                    if label == 1:
                        if weight_assumption > 15:
                            patient_list.append(cur_patient)
                            aki += 1
                            end_time_list.append(time_4)
                            start_time_list.append(time_4 - 39600)
                            label_list.append(label)
                            age_list.append(age)
                            weight_list.append(weight_assumption)
                            base_list.append('u')
                            icu_time_list.append(time_1)
                            leave_time_list.append(time_3)
                    elif label == 0:
                        if weight_assumption > 15:
                            if time_2 - time_1 >= 43200:
                                patient_list.append(cur_patient)
                                no_aki += 1
                                end_time_list.append(time_2 - 3600)
                                start_time_list.append(time_2 - 43200)
                                label_list.append(label)
                                age_list.append(age)
                                weight_list.append(weight_assumption)
                                base_list.append('u')
                                icu_time_list.append(time_1)
                                leave_time_list.append(time_3)
                    ########結束處理 
            ########更新現今處理病人的病歷號
            last_patient = cur_patient
            cur_patient = int(df[num_of_records][3])
            ########把新的病人append進來
            ary_list = []
            ary_list.append(df[num_of_records])
        ########持續append cur_patient的資料
        else:
            #print('append')
            ary_list.append(df[num_of_records])                        
                        
    ########處理最後一個病人
    age = ary_list[0][4]
    weight_assumption = False
    for temp in ary_list:
        if weight_assumption == False and np.isnan(temp[1]) == 0 and temp[1] != 0:
            weight_assumption = temp[1]
            break
            
    ########確定病人有體重紀錄        
    if weight_assumption != False:
        ########清空變數         
        cur_weight_list = []
        cur_urine_list = []
        time_1 = 0
        time_2 = 0
        time_3 = 0
        last_time_3 = 0
        time_4 = 0
        label = 3

        ########初始化變數
        time_1 = ary_list[0][2].timestamp()
        time_2 = time_1

        ########開始label
        for temp in ary_list:        
            time_3 = temp[2].timestamp()
            ########處裡前幾筆資料的label
            if one_hour_pass(time_2, time_3) == 1 and label != 2 and label != 1:
                if len(cur_urine_list) and len(cur_weight_list) != 0:
                    cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
                elif len(cur_urine_list) != 0:
                    cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
                ########超過一小時 但前幾筆資料沒有6小時尿量紀錄
                else:
                    cur_label = False
                ########
                if cur_label != False:
                    if cur_label == 1:
                        if time_2 - time_1 >= 43200:
                            label = 1
                            time_4 = time_2 - 3600
                        else: 
                            label = 2
                    else:
                        label = 0

                ########調整基準時間 計算前後兩筆紀錄是否超過6小時
                if (time_3 - last_time_3 > 21600) and (label != 1) and (label != 2) and (last_time_3 != 0):
                    if time_2 + 21600 - time_1 >= 43200:
                        label = 1
                        time_4 = time_2 + 21600 - 3600
                    else:
                        label = 2
                        
                while time_3 - time_2 >= 3600:
                    time_2 += 3600
             
                ########清空尿量和體重暫存器
                cur_weight_list = []
                cur_urine_list = []
            ########累積尿量及體重資料
            if np.isnan(temp[0]) == 0:
                cur_urine_list.append(temp[0])

            if np.isnan(temp[1]) == 0 and temp[1] != 0:
                cur_weight_list.append(temp[1])
                
            ########儲存這次紀錄的時間 之後和下次比對    
            last_time_3 = time_3
        ########ary_list循環完畢 但urine_list不是空的 => 最後未滿一小時的資料還沒進行label        
        if len(cur_urine_list) != 0 and label != 2 and label != 1:
            if len(cur_urine_list) and len(cur_weight_list) != 0:
                cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
            elif len(cur_urine_list) != 0:
                cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
            ########if anything happens
            else:
                cur_label = False
            ########
            if cur_label != False:
                if cur_label == 1:
                    if time_2 - time_1 >= 43200:
                        label = 1
                        time_4 = time_2 - 3600
                    else: 
                        label = 2
                else:
                    label = 0
    ########單一FILE所有病人處理完畢
    
        ########把該append進去處理好的list的資料append進去
        if label == 1:
            if weight_assumption > 15:
                patient_list.append(cur_patient)
                aki += 1
                end_time_list.append(time_4)
                start_time_list.append(time_4 - 39600)
                label_list.append(label)
                age_list.append(age)
                weight_list.append(weight_assumption)
                base_list.append('u')
                icu_time_list.append(time_1)
                leave_time_list.append(time_3)
        elif label == 0:
            if weight_assumption > 15:
                if time_2 - time_1 >= 43200:
                    patient_list.append(cur_patient)
                    no_aki += 1
                    end_time_list.append(time_2 - 3600)
                    start_time_list.append(time_2 - 43200)
                    label_list.append(label)
                    age_list.append(age)
                    weight_list.append(weight_assumption)
                    base_list.append('u')
                    icu_time_list.append(time_1)
                    leave_time_list.append(time_3)
            
    ########整理小list    
    patient_list.pop(0)
    label_list.pop(0)
    start_time_list.pop(0)
    end_time_list.pop(0)
    age_list.pop(0)
    weight_list.pop(0)
    base_list.pop(0)
    icu_time_list.pop(0)
    leave_time_list.pop(0)
    
    ########放進大list
    for value in patient_list:
        big_patient_list.append(value)
    for value in label_list:
        big_label_list.append(value)
    for value in start_time_list:
        big_start_time_list.append(value)
    for value in end_time_list:
        big_end_time_list.append(value)
    for value in age_list:
        big_age_list.append(value)
    for value in weight_list:
        big_weight_list.append(value)
    for value in base_list:
        big_base_list.append(value)
    for value in icu_time_list:
        big_icu_time_list.append(value)
    for value in leave_time_list:
        big_leave_time_list.append(value)
        
    #label的結果
    #print(patient_list)        
    #print(label_list)
    #print(len(patient_list))
    #print(len(label_list))
    #print(FILE)
    #print(age_list)
#     break
# print(df)
    
print(aki)
print(no_aki)

In [ ]:
##########################################################################################
#                                   //                                                   #
#                                  //                                                    #
#                                 //////                                                 #
#                                //    //                                                #
#                                ///////                                                 #                                                          #
##########################################################################################
aki = 0
no_aki = 0
month = 1
for FILE in tqdm(urine_filelist):
#     print(month)
    month += 1
    df = pd.read_excel(urine_path + FILE, sheet_name = "匯出資料", usecols = ["6小時總尿量", "尿液輸出前最新體重", "尿液輸出時間", "病歷號", "年齡"]) 
    #6小時總尿量, 體重, 輸出時間, 病歷號 年齡
    df = df.to_numpy()
    patient_list = [0]
    label_list = [0] 
    index = 0
    weight_list = [0]
    cur_weight_list = []
    cur_urine_list = []
    start_time_list = [0]
    end_time_list = [0]
    age_list = [0]
    base_list = [0]
    icu_time_list = [0]
    leave_time_list = [0]
    label = 3 #label: 0 沒有AKI; 1 有AKI, 2 入院23小時內就得AKI; 3 資料有缺失
    time_1 = 0 #進ICU的時間
    time_2 = 0 #一個小時的起始點
    time_3 = 0 #當前紀錄的時間點
    time_4 = 0 #重要的時間點 label 0 不會用到 label 1 等於發病時間前一小時
    last_patient = 0
    cur_patient = 0
    ary_list = []
    weight_assumption = False
    age = False
    
    for num_of_records in range(0,df.shape[0]):
        ########碰到x病人就直接跳過
        if str.isdigit(str(df[num_of_records][3])) == False:
            continue
        ########碰到年齡有問題的病人也跳過
        elif (df[num_of_records][4] < 18 or np.isnan(df[num_of_records][4]) == 1):
            continue
        ########碰到新的病人
        elif int(df[num_of_records][3]) != cur_patient:
            #print(cur_patient)
        ########先處裡舊的 df的第一個病人不會有舊的病人可以處理    
            if cur_patient != last_patient:
                #print('process')
                ########開始處理
                age = ary_list[0][4]
                ########先取得病人最早的體重紀錄 以防有尿量紀錄卻無體重紀錄的狀況
                weight_assumption = False
                #print(ary_list)
                for temp in ary_list:
                    if weight_assumption == False and np.isnan(temp[1]) == 0 and temp[1] != 0:
                        weight_assumption = temp[1]
                        break
                        
                ########確定病人有體重紀錄
                if weight_assumption != False:
                    ########清空變數         
                    cur_weight_list = []
                    cur_urine_list = []
                    time_1 = 0
                    time_2 = 0
                    time_3 = 0
                    last_time_3 = 0
                    time_4 = 0
                    label = 3

                    ########初始化變數
                    time_1 = ary_list[0][2].timestamp()
                    time_2 = time_1
                    ########開始label
                    #print('start_label')
                    for temp in ary_list:        
                        time_3 = temp[2].timestamp()
                        cur_label = False
                        ########處裡前幾筆資料的label
                        if one_hour_pass(time_2, time_3) == 1 and label != 2 and label != 1:
                            #print('start_label')
                            if len(cur_urine_list) and len(cur_weight_list) != 0:
                                cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
                            elif len(cur_urine_list) != 0:
                                cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
                            ########超過一小時 但前幾筆資料沒有6小時尿量紀錄
                            else:
                                cur_label = False
                            #print(cur_label)
                            ########
                            if cur_label != False:
                                if cur_label == 1:
                                    if time_2 - time_1 >= 21600:
                                        label = 1
                                        time_4 = time_2 - 3600
                                    else: 
                                        label = 2
                                else:
                                    label = 0
                                #print(label)

                            ########調整基準時間 計算前後兩筆紀錄是否超過6小時
                            if (time_3 - last_time_3 > 21600) and (label != 1) and (label != 2) and (last_time_3 != 0):
                                if time_2 + 21600 - time_1 >= 21600:
                                    label = 1
                                    time_4 = time_2 + 21600 - 3600
                                else:
                                    label = 2
                                    
                            while time_3 - time_2 >= 3600:
                                time_2 += 3600        
                                    
                            ########清空尿量和體重暫存器
                            cur_weight_list = []
                            cur_urine_list = []
                        ########累積尿量及體重資料
                        if np.isnan(temp[0]) == 0:
                            cur_urine_list.append(temp[0])

                        if np.isnan(temp[1]) == 0 and temp[1] != 0:
                            cur_weight_list.append(temp[1])

                    ########ary_list循環完畢 但urine_list不是空的 => 最後未滿一小時的資料還沒進行label        
                    if len(cur_urine_list) != 0 and label != 2 and label != 1:
                        if len(cur_urine_list) and len(cur_weight_list) != 0:
                            cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
                        elif len(cur_urine_list) != 0:
                            cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
                        ########if anything happens
                        else:
                            cur_label = False
                        ########
                        if cur_label != False:
                            if cur_label == 1:
                                ######## 86400為24小時
                                # if time_2 - time_1 >= 86400:
                                #     label = 1
                                #     time_4 = time_2 - 3600
                                ######## 43200為12小時
                                if time_2 - time_1 >= 21600:
                                    label = 1
                                    time_4 = time_2 - 3600
                                else: 
                                    label = 2
                            else:
                                label = 0
                    ########把該append進去處理好的list的資料append進去
                    #print(label)
                    if label == 1:
                        if weight_assumption > 15:
                            patient_list.append(cur_patient)
                            aki += 1
                            end_time_list.append(time_4)
                            start_time_list.append(time_4 - 18000)
                            label_list.append(label)
                            age_list.append(age)
                            weight_list.append(weight_assumption)
                            base_list.append('u')
                            icu_time_list.append(time_1)
                            leave_time_list.append(time_3)
                    elif label == 0:
                        if weight_assumption > 15:
                            if time_2 - time_1 >= 21600:
                                patient_list.append(cur_patient)
                                no_aki += 1
                                end_time_list.append(time_2 - 3600)
                                start_time_list.append(time_2 - 21600)
                                label_list.append(label)
                                age_list.append(age)
                                weight_list.append(weight_assumption)
                                base_list.append('u')
                                icu_time_list.append(time_1)
                                leave_time_list.append(time_3)
                    ########結束處理 
            ########更新現今處理病人的病歷號
            last_patient = cur_patient
            cur_patient = int(df[num_of_records][3])
            ########把新的病人append進來
            ary_list = []
            ary_list.append(df[num_of_records])
        ########持續append cur_patient的資料
        else:
            #print('append')
            ary_list.append(df[num_of_records])                        
                        
    ########處理最後一個病人
    age = ary_list[0][4]
    weight_assumption = False
    for temp in ary_list:
        if weight_assumption == False and np.isnan(temp[1]) == 0 and temp[1] != 0:
            weight_assumption = temp[1]
            break
            
    ########確定病人有體重紀錄        
    if weight_assumption != False:
        ########清空變數         
        cur_weight_list = []
        cur_urine_list = []
        time_1 = 0
        time_2 = 0
        time_3 = 0
        last_time_3 = 0
        time_4 = 0
        label = 3

        ########初始化變數
        time_1 = ary_list[0][2].timestamp()
        time_2 = time_1

        ########開始label
        for temp in ary_list:        
            time_3 = temp[2].timestamp()
            ########處裡前幾筆資料的label
            if one_hour_pass(time_2, time_3) == 1 and label != 2 and label != 1:
                if len(cur_urine_list) and len(cur_weight_list) != 0:
                    cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
                elif len(cur_urine_list) != 0:
                    cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
                ########超過一小時 但前幾筆資料沒有6小時尿量紀錄
                else:
                    cur_label = False
                ########
                if cur_label != False:
                    if cur_label == 1:
                        if time_2 - time_1 >= 21600:
                            label = 1
                            time_4 = time_2 - 3600
                        else: 
                            label = 2
                    else:
                        label = 0

                ########調整基準時間 計算前後兩筆紀錄是否超過6小時
                if (time_3 - last_time_3 > 21600) and (label != 1) and (label != 2) and (last_time_3 != 0):
                    if time_2 + 21600 - time_1 >= 21600:
                        label = 1
                        time_4 = time_2 + 21600 - 3600
                    else:
                        label = 2
                        
                while time_3 - time_2 >= 3600:
                    time_2 += 3600
             
                ########清空尿量和體重暫存器
                cur_weight_list = []
                cur_urine_list = []
            ########累積尿量及體重資料
            if np.isnan(temp[0]) == 0:
                cur_urine_list.append(temp[0])

            if np.isnan(temp[1]) == 0 and temp[1] != 0:
                cur_weight_list.append(temp[1])
                
            ########儲存這次紀錄的時間 之後和下次比對    
            last_time_3 = time_3
        ########ary_list循環完畢 但urine_list不是空的 => 最後未滿一小時的資料還沒進行label        
        if len(cur_urine_list) != 0 and label != 2 and label != 1:
            if len(cur_urine_list) and len(cur_weight_list) != 0:
                cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
            elif len(cur_urine_list) != 0:
                cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
            ########if anything happens
            else:
                cur_label = False
            ########
            if cur_label != False:
                if cur_label == 1:
                    if time_2 - time_1 >= 21600:
                        label = 1
                        time_4 = time_2 - 3600
                    else: 
                        label = 2
                else:
                    label = 0
    ########單一FILE所有病人處理完畢
    
        ########把該append進去處理好的list的資料append進去
        if label == 1:
            if weight_assumption > 15:
                patient_list.append(cur_patient)
                aki += 1
                end_time_list.append(time_4)
                start_time_list.append(time_4 - 39600)
                label_list.append(label)
                age_list.append(age)
                weight_list.append(weight_assumption)
                base_list.append('u')
                icu_time_list.append(time_1)
                leave_time_list.append(time_3)
        elif label == 0:
            if weight_assumption > 15:
                if time_2 - time_1 >= 21600:
                    patient_list.append(cur_patient)
                    no_aki += 1
                    end_time_list.append(time_2 - 3600)
                    start_time_list.append(time_2 - 21600)
                    label_list.append(label)
                    age_list.append(age)
                    weight_list.append(weight_assumption)
                    base_list.append('u')
                    icu_time_list.append(time_1)
                    leave_time_list.append(time_3)
            
    ########整理小list    
    patient_list.pop(0)
    label_list.pop(0)
    start_time_list.pop(0)
    end_time_list.pop(0)
    age_list.pop(0)
    weight_list.pop(0)
    base_list.pop(0)
    icu_time_list.pop(0)
    leave_time_list.pop(0)
    
    ########放進大list
    for value in patient_list:
        big_patient_list.append(value)
    for value in label_list:
        big_label_list.append(value)
    for value in start_time_list:
        big_start_time_list.append(value)
    for value in end_time_list:
        big_end_time_list.append(value)
    for value in age_list:
        big_age_list.append(value)
    for value in weight_list:
        big_weight_list.append(value)
    for value in base_list:
        big_base_list.append(value)
    for value in icu_time_list:
        big_icu_time_list.append(value)
    for value in leave_time_list:
        big_leave_time_list.append(value)
        
    #label的結果
    #print(patient_list)        
    #print(label_list)
    #print(len(patient_list))
    #print(len(label_list))
    #print(FILE)
    #print(age_list)
#     break
# print(df)
    
print(aki)
print(no_aki)

# 修改完畢

In [ ]:
##########################################################################################
#                            //////        //                                            #
#                          //   //        //                                             #
#                             //         //  //                                          #
#                           //          /////////                                        #
#                         ////////          //                                           #
##########################################################################################
aki = 0
no_aki = 0
month = 1
for FILE in tqdm(urine_filelist):
#     print(month)
    month += 1
    df = pd.read_excel(urine_path + FILE, sheet_name = "匯出資料", usecols = ["6小時總尿量", "尿液輸出前最新體重", "尿液輸出時間", "病歷號", "年齡"]) 
    #6小時總尿量, 體重, 輸出時間, 病歷號 年齡
    df = df.to_numpy()
    patient_list = [0]
    label_list = [0] 
    index = 0
    weight_list = [0]
    cur_weight_list = []
    cur_urine_list = []
    start_time_list = [0]
    end_time_list = [0]
    age_list = [0]
    base_list = [0]
    icu_time_list = [0]
    leave_time_list = [0]
    label = 3 #label: 0 沒有AKI; 1 有AKI, 2 入院23小時內就得AKI; 3 資料有缺失
    time_1 = 0 #進ICU的時間
    time_2 = 0 #一個小時的起始點
    time_3 = 0 #當前紀錄的時間點
    time_4 = 0 #重要的時間點 label 0 不會用到 label 1 等於發病時間前一小時
    last_patient = 0
    cur_patient = 0
    ary_list = []
    weight_assumption = False
    age = False
    
    for num_of_records in range(0,df.shape[0]):
        ########碰到x病人就直接跳過
        if str.isdigit(str(df[num_of_records][3])) == False:
            continue
        ########碰到年齡有問題的病人也跳過
        elif (df[num_of_records][4] < 18 or np.isnan(df[num_of_records][4]) == 1):
            continue
        ########碰到新的病人
        elif int(df[num_of_records][3]) != cur_patient:
            #print(cur_patient)
        ########先處裡舊的 df的第一個病人不會有舊的病人可以處理    
            if cur_patient != last_patient:
                #print('process')
                ########開始處理
                age = ary_list[0][4]
                ########先取得病人最早的體重紀錄 以防有尿量紀錄卻無體重紀錄的狀況
                weight_assumption = False
                #print(ary_list)
                for temp in ary_list:
                    if weight_assumption == False and np.isnan(temp[1]) == 0 and temp[1] != 0:
                        weight_assumption = temp[1]
                        break
                        
                ########確定病人有體重紀錄
                if weight_assumption != False:
                    ########清空變數         
                    cur_weight_list = []
                    cur_urine_list = []
                    time_1 = 0
                    time_2 = 0
                    time_3 = 0
                    last_time_3 = 0
                    time_4 = 0
                    label = 3

                    ########初始化變數
                    time_1 = ary_list[0][2].timestamp()
                    time_2 = time_1
                    
                    ########開始label
                    for temp in ary_list:        
                        time_3 = temp[2].timestamp()
                        cur_label = False
                        
                        ########處裡前幾筆資料的label
                        if one_hour_pass(time_2, time_3) == 1 and label != 2 and label != 1:
                            if len(cur_urine_list) and len(cur_weight_list) != 0:
                                cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
                            elif len(cur_urine_list) != 0:
                                cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
                            ########超過一小時 但前幾筆資料沒有6小時尿量紀錄
                            else:
                                cur_label = False
                            #print(cur_label)
                            ########
                            if cur_label != False:
                                if cur_label == 1:
                                    if time_2 - time_1 >= 82800:
                                        label = 1
                                        time_4 = time_2
                                    else: 
                                        label = 2
                                else:
                                    label = 0
                                #print(label)

                            ########調整基準時間 計算前後兩筆紀錄是否超過6小時
                            if (time_3 - last_time_3 > 21600) and (label != 1) and (label != 2) and (last_time_3 != 0):
                                if time_2 + 21600 - time_1 >= 82800:
                                    label = 1
                                    time_4 = time_2 + 21600 
                                else:
                                    label = 2
                                    
                            while time_3 - time_2 >= 3600:
                                time_2 += 3600        
                                    
                            ########清空尿量和體重暫存器
                            cur_weight_list = []
                            cur_urine_list = []
                        ########累積尿量及體重資料
                        if np.isnan(temp[0]) == 0:
                            cur_urine_list.append(temp[0])

                        if np.isnan(temp[1]) == 0 and temp[1] != 0:
                            cur_weight_list.append(temp[1])
                            
                        ########儲存這次紀錄的時間 之後和下次比對    
                        last_time_3 = time_3    

                    ########ary_list循環完畢 但urine_list不是空的 => 最後未滿一小時的資料還沒進行label        
                    if len(cur_urine_list) != 0 and label != 2 and label != 1:
                        if len(cur_urine_list) and len(cur_weight_list) != 0:
                            cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
                        elif len(cur_urine_list) != 0:
                            cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
                        ########if anything happens
                        else:
                            cur_label = False
                        ########
                        if cur_label != False:
                            if cur_label == 1:
                                ######## 86400為24小時
                                # if time_2 - time_1 >= 86400:
                                #     label = 1
                                #     time_4 = time_2 - 3600
                                ######## 43200為12小時
                                if time_2 - time_1 >= 82800:
                                    label = 1
                                    time_4 = time_2 
                                else: 
                                    label = 2
                            else:
                                label = 0
                    ########把該append進去處理好的list的資料append進去
                    #print(label)
                    if label == 1:
                        if weight_assumption > 15:
                            patient_list.append(cur_patient)
                            aki += 1
                            end_time_list.append(time_4)
                            start_time_list.append(time_4 - 82800)
                            label_list.append(label)
                            age_list.append(age)
                            weight_list.append(weight_assumption)
                            base_list.append('u')
                            icu_time_list.append(time_1)
                            leave_time_list.append(time_3)
                    elif label == 0:
                        if weight_assumption > 15:
                            if time_2 - time_1 >= 82800:
                                patient_list.append(cur_patient)
                                no_aki += 1
                                end_time_list.append(time_2)
                                start_time_list.append(time_2 - 82800)
                                label_list.append(label)
                                age_list.append(age)
                                weight_list.append(weight_assumption)
                                base_list.append('u')
                                icu_time_list.append(time_1)
                                leave_time_list.append(time_3)
                    ########結束處理 
            ########更新現今處理病人的病歷號
            last_patient = cur_patient
            cur_patient = int(df[num_of_records][3])
            ########把新的病人append進來
            ary_list = []
            ary_list.append(df[num_of_records])
        ########持續append cur_patient的資料
        else:
            #print('append')
            ary_list.append(df[num_of_records])                        
                        
    ########處理最後一個病人
    age = ary_list[0][4]
    weight_assumption = False
    for temp in ary_list:
        if weight_assumption == False and np.isnan(temp[1]) == 0 and temp[1] != 0:
            weight_assumption = temp[1]
            break
            
    ########確定病人有體重紀錄        
    if weight_assumption != False:
        ########清空變數         
        cur_weight_list = []
        cur_urine_list = []
        time_1 = 0
        time_2 = 0
        time_3 = 0
        last_time_3 = 0
        time_4 = 0
        label = 3

        ########初始化變數
        time_1 = ary_list[0][2].timestamp()
        time_2 = time_1

        ########開始label
        for temp in ary_list:        
            time_3 = temp[2].timestamp()
            ########處裡前幾筆資料的label
            if one_hour_pass(time_2, time_3) == 1 and label != 2 and label != 1:
                if len(cur_urine_list) and len(cur_weight_list) != 0:
                    cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
                elif len(cur_urine_list) != 0:
                    cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
                ########超過一小時 但前幾筆資料沒有6小時尿量紀錄
                else:
                    cur_label = False
                ########
                if cur_label != False:
                    if cur_label == 1:
                        if time_2 - time_1 >= 82800:
                            label = 1
                            time_4 = time_2
                        else: 
                            label = 2
                    else:
                        label = 0

                ########調整基準時間 計算前後兩筆紀錄是否超過6小時
                if (time_3 - last_time_3 > 21600) and (label != 1) and (label != 2) and (last_time_3 != 0):
                    if time_2 + 21600 - time_1 >= 82800:
                        label = 1
                        time_4 = time_2 + 21600 
                    else:
                        label = 2
                        
                while time_3 - time_2 >= 3600:
                    time_2 += 3600
             
                ########清空尿量和體重暫存器
                cur_weight_list = []
                cur_urine_list = []
            ########累積尿量及體重資料
            if np.isnan(temp[0]) == 0:
                cur_urine_list.append(temp[0])

            if np.isnan(temp[1]) == 0 and temp[1] != 0:
                cur_weight_list.append(temp[1])
                
            ########儲存這次紀錄的時間 之後和下次比對    
            last_time_3 = time_3
        ########ary_list循環完畢 但urine_list不是空的 => 最後未滿一小時的資料還沒進行label        
        if len(cur_urine_list) != 0 and label != 2 and label != 1:
            if len(cur_urine_list) and len(cur_weight_list) != 0:
                cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
            elif len(cur_urine_list) != 0:
                cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
            ########if anything happens
            else:
                cur_label = False
            ########
            if cur_label != False:
                if cur_label == 1:
                    if time_2 - time_1 >= 82800:
                        label = 1
                        time_4 = time_2 - 3600
                    else: 
                        label = 2
                else:
                    label = 0
    ########單一FILE所有病人處理完畢
    
        ########把該append進去處理好的list的資料append進去
        if label == 1:
            if weight_assumption > 15:
                patient_list.append(cur_patient)
                aki += 1
                end_time_list.append(time_4)
                start_time_list.append(time_4 - 82800)
                label_list.append(label)
                age_list.append(age)
                weight_list.append(weight_assumption)
                base_list.append('u')
                icu_time_list.append(time_1)
                leave_time_list.append(time_3)
        elif label == 0:
            if weight_assumption > 15:
                if time_2 - time_1 >= 82800:
                    patient_list.append(cur_patient)
                    no_aki += 1
                    end_time_list.append(time_2)
                    start_time_list.append(time_2 - 82800)
                    label_list.append(label)
                    age_list.append(age)
                    weight_list.append(weight_assumption)
                    base_list.append('u')
                    icu_time_list.append(time_1)
                    leave_time_list.append(time_3)
            
    ########整理小list    
    patient_list.pop(0)
    label_list.pop(0)
    start_time_list.pop(0)
    end_time_list.pop(0)
    age_list.pop(0)
    weight_list.pop(0)
    base_list.pop(0)
    icu_time_list.pop(0)
    leave_time_list.pop(0)
    
    ########放進大list
    for value in patient_list:
        big_patient_list.append(value)
    for value in label_list:
        big_label_list.append(value)
    for value in start_time_list:
        big_start_time_list.append(value)
    for value in end_time_list:
        big_end_time_list.append(value)
    for value in age_list:
        big_age_list.append(value)
    for value in weight_list:
        big_weight_list.append(value)
    for value in base_list:
        big_base_list.append(value)
    for value in icu_time_list:
        big_icu_time_list.append(value)
    for value in leave_time_list:
        big_leave_time_list.append(value)
        
    #label的結果
    #print(patient_list)        
    #print(label_list)
    #print(len(patient_list))
    #print(len(label_list))
    #print(FILE)
    #print(age_list)
#     break
# print(df)
    
print(aki)
print(no_aki)

In [ ]:
## 18

aki = 0
no_aki = 0
month = 1
for FILE in tqdm(urine_filelist):
#     print(month)
    month += 1
    df = pd.read_excel(urine_path + FILE, sheet_name = "匯出資料", usecols = ["6小時總尿量", "尿液輸出前最新體重", "尿液輸出時間", "病歷號", "年齡"]) 
    #6小時總尿量, 體重, 輸出時間, 病歷號 年齡
    df = df.to_numpy()
    patient_list = [0]
    label_list = [0] 
    index = 0
    weight_list = [0]
    cur_weight_list = []
    cur_urine_list = []
    start_time_list = [0]
    end_time_list = [0]
    age_list = [0]
    base_list = [0]
    icu_time_list = [0]
    leave_time_list = [0]
    label = 3 #label: 0 沒有AKI; 1 有AKI, 2 入院23小時內就得AKI; 3 資料有缺失
    time_1 = 0 #進ICU的時間
    time_2 = 0 #一個小時的起始點
    time_3 = 0 #當前紀錄的時間點
    time_4 = 0 #重要的時間點 label 0 不會用到 label 1 等於發病時間前一小時
    last_patient = 0
    cur_patient = 0
    ary_list = []
    weight_assumption = False
    age = False
    
    for num_of_records in range(0,df.shape[0]):
        ########碰到x病人就直接跳過
        if str.isdigit(str(df[num_of_records][3])) == False:
            continue
        ########碰到年齡有問題的病人也跳過
        elif (df[num_of_records][4] < 18 or np.isnan(df[num_of_records][4]) == 1):
            continue
        ########碰到新的病人
        elif int(df[num_of_records][3]) != cur_patient:
            #print(cur_patient)
        ########先處裡舊的 df的第一個病人不會有舊的病人可以處理    
            if cur_patient != last_patient:
                #print('process')
                ########開始處理
                age = ary_list[0][4]
                ########先取得病人最早的體重紀錄 以防有尿量紀錄卻無體重紀錄的狀況
                weight_assumption = False
                #print(ary_list)
                for temp in ary_list:
                    if weight_assumption == False and np.isnan(temp[1]) == 0 and temp[1] != 0:
                        weight_assumption = temp[1]
                        break
                        
                ########確定病人有體重紀錄
                if weight_assumption != False:
                    ########清空變數         
                    cur_weight_list = []
                    cur_urine_list = []
                    time_1 = 0
                    time_2 = 0
                    time_3 = 0
                    last_time_3 = 0
                    time_4 = 0
                    label = 3

                    ########初始化變數
                    time_1 = ary_list[0][2].timestamp()
                    time_2 = time_1
                    
                    ########開始label
                    for temp in ary_list:        
                        time_3 = temp[2].timestamp()
                        cur_label = False
                        
                        ########處裡前幾筆資料的label
                        if one_hour_pass(time_2, time_3) == 1 and label != 2 and label != 1:
                            if len(cur_urine_list) and len(cur_weight_list) != 0:
                                cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
                            elif len(cur_urine_list) != 0:
                                cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
                            ########超過一小時 但前幾筆資料沒有6小時尿量紀錄
                            else:
                                cur_label = False
                            #print(cur_label)
                            ########
                            if cur_label != False:
                                if cur_label == 1:
                                    if time_2 - time_1 >= 61200:
                                        label = 1
                                        time_4 = time_2
                                    else: 
                                        label = 2
                                else:
                                    label = 0
                                #print(label)

                            ########調整基準時間 計算前後兩筆紀錄是否超過6小時
                            if (time_3 - last_time_3 > 21600) and (label != 1) and (label != 2) and (last_time_3 != 0):
                                if time_2 + 21600 - time_1 >= 61200:
                                    label = 1
                                    time_4 = time_2 + 21600 
                                else:
                                    label = 2
                                    
                            while time_3 - time_2 >= 3600:
                                time_2 += 3600        
                                    
                            ########清空尿量和體重暫存器
                            cur_weight_list = []
                            cur_urine_list = []
                        ########累積尿量及體重資料
                        if np.isnan(temp[0]) == 0:
                            cur_urine_list.append(temp[0])

                        if np.isnan(temp[1]) == 0 and temp[1] != 0:
                            cur_weight_list.append(temp[1])
                            
                        ########儲存這次紀錄的時間 之後和下次比對    
                        last_time_3 = time_3    

                    ########ary_list循環完畢 但urine_list不是空的 => 最後未滿一小時的資料還沒進行label        
                    if len(cur_urine_list) != 0 and label != 2 and label != 1:
                        if len(cur_urine_list) and len(cur_weight_list) != 0:
                            cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
                        elif len(cur_urine_list) != 0:
                            cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
                        ########if anything happens
                        else:
                            cur_label = False
                        ########
                        if cur_label != False:
                            if cur_label == 1:
                                ######## 86400為24小時
                                # if time_2 - time_1 >= 86400:
                                #     label = 1
                                #     time_4 = time_2 - 3600
                                ######## 43200為12小時
                                if time_2 - time_1 >= 61200:
                                    label = 1
                                    time_4 = time_2 
                                else: 
                                    label = 2
                            else:
                                label = 0
                    ########把該append進去處理好的list的資料append進去
                    #print(label)
                    if label == 1:
                        if weight_assumption > 15:
                            patient_list.append(cur_patient)
                            aki += 1
                            end_time_list.append(time_4)
                            start_time_list.append(time_4 - 61200)
                            label_list.append(label)
                            age_list.append(age)
                            weight_list.append(weight_assumption)
                            base_list.append('u')
                            icu_time_list.append(time_1)
                            leave_time_list.append(time_3)
                    elif label == 0:
                        if weight_assumption > 15:
                            if time_2 - time_1 >= 61200:
                                patient_list.append(cur_patient)
                                no_aki += 1
                                end_time_list.append(time_2)
                                start_time_list.append(time_2 - 61200)
                                label_list.append(label)
                                age_list.append(age)
                                weight_list.append(weight_assumption)
                                base_list.append('u')
                                icu_time_list.append(time_1)
                                leave_time_list.append(time_3)
                    ########結束處理 
            ########更新現今處理病人的病歷號
            last_patient = cur_patient
            cur_patient = int(df[num_of_records][3])
            ########把新的病人append進來
            ary_list = []
            ary_list.append(df[num_of_records])
        ########持續append cur_patient的資料
        else:
            #print('append')
            ary_list.append(df[num_of_records])                        
                        
    ########處理最後一個病人
    age = ary_list[0][4]
    weight_assumption = False
    for temp in ary_list:
        if weight_assumption == False and np.isnan(temp[1]) == 0 and temp[1] != 0:
            weight_assumption = temp[1]
            break
            
    ########確定病人有體重紀錄        
    if weight_assumption != False:
        ########清空變數         
        cur_weight_list = []
        cur_urine_list = []
        time_1 = 0
        time_2 = 0
        time_3 = 0
        last_time_3 = 0
        time_4 = 0
        label = 3

        ########初始化變數
        time_1 = ary_list[0][2].timestamp()
        time_2 = time_1

        ########開始label
        for temp in ary_list:        
            time_3 = temp[2].timestamp()
            ########處裡前幾筆資料的label
            if one_hour_pass(time_2, time_3) == 1 and label != 2 and label != 1:
                if len(cur_urine_list) and len(cur_weight_list) != 0:
                    cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
                elif len(cur_urine_list) != 0:
                    cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
                ########超過一小時 但前幾筆資料沒有6小時尿量紀錄
                else:
                    cur_label = False
                ########
                if cur_label != False:
                    if cur_label == 1:
                        if time_2 - time_1 >= 61200:
                            label = 1
                            time_4 = time_2
                        else: 
                            label = 2
                    else:
                        label = 0

                ########調整基準時間 計算前後兩筆紀錄是否超過6小時
                if (time_3 - last_time_3 > 21600) and (label != 1) and (label != 2) and (last_time_3 != 0):
                    if time_2 + 21600 - time_1 >= 61200:
                        label = 1
                        time_4 = time_2 + 21600 
                    else:
                        label = 2
                        
                while time_3 - time_2 >= 3600:
                    time_2 += 3600
             
                ########清空尿量和體重暫存器
                cur_weight_list = []
                cur_urine_list = []
            ########累積尿量及體重資料
            if np.isnan(temp[0]) == 0:
                cur_urine_list.append(temp[0])

            if np.isnan(temp[1]) == 0 and temp[1] != 0:
                cur_weight_list.append(temp[1])
                
            ########儲存這次紀錄的時間 之後和下次比對    
            last_time_3 = time_3
        ########ary_list循環完畢 但urine_list不是空的 => 最後未滿一小時的資料還沒進行label        
        if len(cur_urine_list) != 0 and label != 2 and label != 1:
            if len(cur_urine_list) and len(cur_weight_list) != 0:
                cur_label = has_aki(list_avg(cur_urine_list), list_avg(cur_weight_list))
            elif len(cur_urine_list) != 0:
                cur_label = has_aki(list_avg(cur_urine_list), weight_assumption)
            ########if anything happens
            else:
                cur_label = False
            ########
            if cur_label != False:
                if cur_label == 1:
                    if time_2 - time_1 >= 61200:
                        label = 1
                        time_4 = time_2 - 3600
                    else: 
                        label = 2
                else:
                    label = 0
    ########單一FILE所有病人處理完畢
    
        ########把該append進去處理好的list的資料append進去
        if label == 1:
            if weight_assumption > 15:
                patient_list.append(cur_patient)
                aki += 1
                end_time_list.append(time_4)
                start_time_list.append(time_4 - 61200)
                label_list.append(label)
                age_list.append(age)
                weight_list.append(weight_assumption)
                base_list.append('u')
                icu_time_list.append(time_1)
                leave_time_list.append(time_3)
        elif label == 0:
            if weight_assumption > 15:
                if time_2 - time_1 >= 61200:
                    patient_list.append(cur_patient)
                    no_aki += 1
                    end_time_list.append(time_2)
                    start_time_list.append(time_2 - 61200)
                    label_list.append(label)
                    age_list.append(age)
                    weight_list.append(weight_assumption)
                    base_list.append('u')
                    icu_time_list.append(time_1)
                    leave_time_list.append(time_3)
            
    ########整理小list    
    patient_list.pop(0)
    label_list.pop(0)
    start_time_list.pop(0)
    end_time_list.pop(0)
    age_list.pop(0)
    weight_list.pop(0)
    base_list.pop(0)
    icu_time_list.pop(0)
    leave_time_list.pop(0)
    
    ########放進大list
    for value in patient_list:
        big_patient_list.append(value)
    for value in label_list:
        big_label_list.append(value)
    for value in start_time_list:
        big_start_time_list.append(value)
    for value in end_time_list:
        big_end_time_list.append(value)
    for value in age_list:
        big_age_list.append(value)
    for value in weight_list:
        big_weight_list.append(value)
    for value in base_list:
        big_base_list.append(value)
    for value in icu_time_list:
        big_icu_time_list.append(value)
    for value in leave_time_list:
        big_leave_time_list.append(value)
        
    #label的結果
    #print(patient_list)        
    #print(label_list)
    #print(len(patient_list))
    #print(len(label_list))
    #print(FILE)
    #print(age_list)
#     break
# print(df)
    
print(aki)
print(no_aki)